In [1]:
%%writefile streamlit_app.py
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
import plotly.express as px
import plotly.graph_objects as go
import base64
import time
from PIL import Image
import io
import requests
import gc

# Set page config
st.set_page_config(
    page_title="BCI- EEG signal based controller",
    page_icon="🧠",
    layout="wide"
)

# Label mappings
label_to_command = {0: "🔴 Backward", 1: "⬅️ Left", 2: "➡️ Right", 3: "🟢 Forward"}
emotion_labels = {0: "😠 NEGATIVE", 1: "😐 NEUTRAL", 2: "😊 POSITIVE"}
emotion_colors = {0: "#ff4b4b", 1: "#a3a3a3", 2: "#4bb543"}
robotic_arm_commands = {0: "👊 Grip", 1: "✋ Release", 2: "↪️ Rotate Left", 3: "↩️ Rotate Right"}

# Load the models and scalers
@st.cache_resource
def load_models():
    try:
        with open('best_eeg_model.joblib', 'rb') as f:
            eeg_model = joblib.load(f)

        with open('eeg_scaler.joblib', 'rb') as f:
            eeg_scaler = joblib.load(f)

        with open('best_emotion_model.joblib', 'rb') as f:
            emotion_model = joblib.load(f)

        with open('emotion_scaler.joblib', 'rb') as f:
            emotion_scaler = joblib.load(f)

        with open('emotion_feature_selector.joblib', 'rb') as f:
            feature_selector = joblib.load(f)

        gc.collect()
        return eeg_model, eeg_scaler, emotion_model, emotion_scaler, feature_selector
    except Exception as e:
        st.error(f"Error loading models: {str(e)}")
        return None, None, None, None, None

# Function to load EEG sensor positions image
@st.cache_resource
def load_eeg_sensor_image():
    try:
        response = requests.get("https://hebbkx1anhila5yf.public.blob.vercel-storage.com/EEG_sensor_positions-4JF0gqWXRy6wxFkLC1FqD3SouEGsjw.png")
        img = Image.open(io.BytesIO(response.content))
        return img
    except Exception as e:
        st.error(f"Error loading EEG sensor image: {str(e)}")
        return None

# Function to preprocess data for EEG
def preprocess_eeg_data(df):
    columns_to_drop = ['Start Timestamp', 'End Timestamp', 'Label']
    columns_to_drop = [col for col in columns_to_drop if col in df.columns]
    X = df.drop(columns=columns_to_drop)
    return X

# Function to preprocess data for emotion detection with memory optimization
def preprocess_emotion_data(df):
    try:
        X = df.copy()
        if 'Label' in X.columns:
            X = X.drop('Label', axis=1)
        X = X.astype(np.float32)
        return X
    except Exception as e:
        st.error(f"Error preprocessing data: {str(e)}")
        return None

# Function to make EEG predictions
def make_eeg_predictions(model, scaler, X):
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    return predictions

# Function to make emotion predictions with memory optimization
def make_emotion_predictions(model, scaler, feature_selector, X):
    try:
        batch_size = 16
        predictions = []
        model_features = 489

        for i in range(0, len(X), batch_size):
            batch = X.iloc[i:i+batch_size]
            batch_array = batch.values
            batch_scaled = scaler.transform(batch_array)

            try:
                batch_features = feature_selector.transform(batch_scaled)
            except Exception as selector_error:
                print(f"Feature selector error: {str(selector_error)}")
                if batch_scaled.shape[1] > model_features:
                    batch_features = batch_scaled[:, :model_features]
                else:
                    st.error(f"Model expects {model_features} features but input has only {batch_scaled.shape[1]}")
                    return None

            batch_pred = model.predict_proba(batch_features)
            batch_classes = np.argmax(batch_pred, axis=1)
            predictions.extend(batch_classes)
            del batch
            gc.collect()

        return np.array(predictions)
    except Exception as e:
        st.error(f"Error making emotion predictions: {str(e)}")
        print(f"Exception details: {type(e).__name__}: {str(e)}")
        return None

# Function to make predictions
def make_predictions(model, scaler, X):
    X_scaled = scaler.transform(X)
    predictions = model.predict(X_scaled)
    return predictions

# Function to create download link
def get_table_download_link(df, filename):
    csv = df.to_csv(index=False)
    b64 = base64.b64encode(csv.encode()).decode()
    href = f'<a href="data:file/csv;base64,{b64}" download="{filename}">Download {filename}</a>'
    return href

# Function to create emotion gauge chart
def create_emotion_gauge(emotion_counts):
    total = sum(emotion_counts.values())
    fig = go.Figure()

    for emotion, count in emotion_counts.items():
        percentage = (count / total) * 100 if total > 0 else 0
        fig.add_trace(go.Indicator(
            mode="gauge+number",
            value=percentage,
            title={'text': emotion_labels[emotion]},
            domain={'x': [0.1, 0.9], 'y': [0.1 + (emotion * 0.3), 0.3 + (emotion * 0.3)]},
            gauge={
                'axis': {'range': [0, 100]},
                'bar': {'color': emotion_colors[emotion]},
                'bgcolor': "white",
                'borderwidth': 2,
                'bordercolor': "gray",
            }
        ))

    fig.update_layout(
        height=600,
        margin=dict(l=50, r=50, t=30, b=30),
        showlegend=False,
        paper_bgcolor="white",
        font={'size': 16}
    )
    return fig

# Main app
def main():
    st.title("🧠 BCI- EEG signal based controller")
    st.write("This app classifies EEG signals and detects emotions using trained models.")

    # Load models
    eeg_model, eeg_scaler, emotion_model, emotion_scaler, feature_selector = load_models()

    if eeg_model is None or eeg_scaler is None or emotion_model is None:
        st.error("Failed to load one or more models. Please check the model files and try again.")
        return

    # Load EEG sensor positions image
    eeg_sensor_image = load_eeg_sensor_image()

    # Add custom CSS for button styles and new UI elements
    st.markdown("""
    <style>
        .stButton > button {
            width: 100%;
            height: 60px;
            font-size: 1.2em;
            border-radius: 8px;
            border: none;
            cursor: pointer;
            background-color: #ffffff !important;
            border: 1px solid #ccc;
        }
        .stButton > button:hover {
            background-color: #0056b3;
        }
        .emotion-box {
            padding: 20px;
            border-radius: 10px;
            margin: 10px 0;
            text-align: center;
            font-size: 1.2em;
        }
        .error-message {
            color: #ff4b4b;
            padding: 10px;
            border-radius: 5px;
            background-color: #ff4b4b20;
            margin: 10px 0;
        }
        .robotic-arm-box {
            background-color: #f0f4f8;
            border-radius: 15px;
            padding: 20px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            text-align: center;
            height: 100%;
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
        }
        .robotic-arm-icon {
            font-size: 3em;
            margin-bottom: 10px;
        }
        .command-text {
            font-size: 1.5em;
            font-weight: bold;
            color: #2c3e50;
        }
    </style>
    """, unsafe_allow_html=True)

    # Create tabs for different functionalities
    tab1, tab2, tab3, tab4, tab5, tab6 = st.tabs([
        "Wheel Chair EEG Batch Prediction",
        "Wheel Chair Single Signal Prediction",
        "Emotion EEG Batch Prediction",
        "Emotion Single Signal Prediction",
        "Robotic Arm EEG Batch Prediction",
        "Robotic Arm Single Signal Prediction"
    ])

    with tab1:
        batch_cols = st.columns([7, 3])

        with batch_cols[0]:
            st.header("Wheel Chair EEG Batch Prediction")
            st.write("Upload a CSV file containing EEG signals for batch prediction.")

            st.markdown("### Sample Data")
            try:
                sample_data = pd.read_csv('sample_batch_data.csv')
                st.markdown(get_table_download_link(sample_data, 'sample_batch_data.csv'), unsafe_allow_html=True)
            except Exception as e:
                st.error(f"Error loading sample data: {str(e)}")

            uploaded_file = st.file_uploader("Choose a CSV file for EEG", type="csv", key="eeg_uploader")

            if uploaded_file is not None:
                try:
                    df = pd.read_csv(uploaded_file)
                    st.write("Preview of uploaded data:")
                    st.dataframe(df.head())

                    X = preprocess_eeg_data(df)
                    if st.button("Make EEG Predictions"):
                        with st.spinner("Processing EEG signals..."):
                            predictions = make_eeg_predictions(eeg_model, eeg_scaler, X)

                            if predictions is not None:
                                results_df = df.copy()
                                results_df['Predicted_Label'] = predictions
                                results_df['Command'] = results_df['Predicted_Label'].map(label_to_command)

                                st.subheader("EEG Prediction Results")
                                st.dataframe(results_df)

                                csv = results_df.to_csv(index=False)
                                st.download_button(
                                    label="Download EEG Predictions",
                                    data=csv,
                                    file_name='eeg_predictions.csv',
                                    mime='text/csv'
                                )

                                st.subheader("Prediction Distribution")
                                fig = px.histogram(results_df, x='Command',
                                                title='Distribution of Predicted Commands')
                                st.plotly_chart(fig)
                            else:
                                st.error("Failed to make predictions. Please try again.")

                except Exception as e:
                    st.error(f"Error processing file: {str(e)}")

        with batch_cols[1]:
            st.markdown("""
            <h2 style="font-size: 1.8rem; font-weight: 600; margin-bottom: 0.5rem;">EEG Sensor Positions</h2>
            """, unsafe_allow_html=True)

            with st.container():
                st.markdown("""
                <div class="sensor-container">
                    <div class="sensor-subtitle">International 10-20 System</div>
                """, unsafe_allow_html=True)

                if eeg_sensor_image:
                    st.image(
                        eeg_sensor_image,
                        output_format="PNG",
                        width=250
                    )

                st.markdown("""
                    <div class="sensor-description">
                        Key positions shown: Frontal (F), Central (C), Parietal (P), Occipital (O), and Temporal (T) regions
                    </div>
                </div>
                """, unsafe_allow_html=True)

                with st.expander("About EEG Sensors"):
                    st.markdown("""
                    ### Frontal Region
                    - **Fp1, Fp2**: Prefrontal cortex (attention, planning)
                    - **F3, F4**: Frontal lobe (motor planning, emotional regulation)

                    ### Central Region
                    - **C1, C2**: Central region (fine motor control)
                    - **C3, C4**: Central region (sensorimotor functions)
                    - **Cz**: Central midline (motor coordination, supplementary motor area)

                    ### Centro-Parietal Region
                    - **CP1, CP2**: Transition between central and parietal regions
                    - **CP3, CP4**: Sensorimotor integration areas

                    ### Parietal Region
                    - **P3, P4**: Parietal lobe (sensory integration)
                    - **Pz**: Parietal midline (attention, perception)

                    ### Occipital Region
                    - **O1, O2**: Occipital lobe (visual processing)

                    The "z" (zero) indicates electrodes placed along the midline of the head.
                    Odd numbers (1, 3) indicate positions on the left hemisphere, while
                    even numbers (2, 4) indicate positions on the right hemisphere.
                    """)

    with tab2:
        st.header("Wheel Chair Single Signal Prediction")

        input_col = st.container()

        with input_col:
            st.write("Click the below buttons to get predictions.")

            col1, col2, col3, col4 = st.columns(4)

            if 'pasted_data' not in st.session_state:
                st.session_state.pasted_data = ""

            with col1:
                if st.button("⬅️"):
                    st.session_state.pasted_data = "12.941,13.925,0.6384,10.47,5.69507750956912,5.730747106617077,-0.6799478080579413,-0.5096076385895788,23.78,161.44000000000003,283.2301102543645,535.8684057938913,122.84759251428356,757.493569136136,253.4486183671476,79.8,131.81576884959443,194.79125209214985,370.9546482454237,59.93189994946399,89.09799810426821,55.24933294773325,24.86027344873845,24.309113916943712,100.42302432769688,90.32092342764348,10.85806718874554,57.31898139333484,47.020354909146,67.64011704806995,18.10636717029908,78.98041804595647,30.3558967577572,59.4786643680694,10.922036293820709,44.5779904652658,29.437622539853944,36.05712785609055,14.347660669690088,5.792537390504529,6.797032729507975,11.944572227237176,2.103552362594324,4.660091628087092,4.62346371107315,5.70391822753298,3.925774688274664,3.700859356470557,4.056848371028937,4.741094330910339,4.563123522756883,4.119646171547212,3.814004389982326,3.890024903357447,3.481816369701896,3.9747004808916495,3.750845093219823,4.061976044838742,3.557365181930646,3.682522269370368,3.3782282674037063,3.768286880229158,3.4267676044531163,3.465072273168813,3.767530754360707,3.0512859705372515,3.563266785166551,3.5229448100731555,3.5044338404928923,3.299954363057375,3.1170446627813537,3.4812779750409564,3.2402856971806333,3.2599130450828198,3.581453526862449,3.6143937199739455,3.451437312299841,3.329495624304338,3.329495624304338,3.451437312299841,3.6143937199739455,3.581453526862449,3.2599130450828198,3.2402856971806333,3.4812779750409564,3.1170446627813537,3.299954363057375,3.5044338404928923,3.5229448100731555,3.563266785166551,3.0512859705372515,3.767530754360707,3.465072273168813,3.4267676044531163,3.768286880229158,3.3782282674037063,3.682522269370368,3.557365181930646,4.061976044838742,3.750845093219823,3.9747004808916495,3.481816369701896,3.890024903357447,3.814004389982326,4.119646171547212,4.563123522756883,4.741094330910339,4.056848371028937,3.700859356470557,3.925774688274664,5.70391822753298,4.62346371107315,4.660091628087092,2.103552362594324,11.944572227237176,6.797032729507975,5.792537390504529,14.347660669690088,36.05712785609055,29.437622539853944,44.5779904652658,10.922036293820709,59.4786643680694,30.3558967577572,78.98041804595647,18.10636717029908,67.64011704806995,47.020354909146,57.31898139333484,10.85806718874554,90.32092342764348,100.42302432769688,24.309113943712,24.86027344873845,55.24933294773325,89.09799810426821,59.93189994946399,370.9546482454237,194.79125209214985,131.81576884959443"

            with col2:
                if st.button("➡️"):
                    st.session_state.pasted_data = "17.395,18.388,0.42024,5.22,1.6131343224914656,1.6669745049040192,0.5025110070067309,0.239533240624632,9.1,92.08,165.4295971944718,309.57432302160333,58.24719610414238,84.92850314212005,172.0875267674643,52.53,4.349501719306558,4.28840771540847,39.862175538162425,36.4284181692426,30.46551890972791,25.01970495571268,46.3652031959139,33.808681536867226,38.296132245945,12.626765084859084,49.2025937833613,31.4954245434392,52.86057975123696,24.55065196899352,23.27063818626752,32.13744783420577,21.91764509090947,23.78136989731783,15.568329716878544,8.14584129140285,14.028810047107587,25.893231464828432,7.516908722403525,2.595273725776378,5.048236693896485,6.387403956953162,4.361018630229035,2.177201992767261,5.341293038851984,2.497016468137817,3.344512943382261,4.280914528765986,1.9053203907837477,2.2983513828834825,1.3831502596286656,2.00990638700909,1.5212893856644114,2.0075263733257755,2.1062263604341616,1.6701857137564855,1.4578214827166478,1.9360366404920484,1.5368561570693156,1.735988854665956,1.5867914113761434,1.584388961931582,1.7926898224178711,1.4874799944259034,2.1135601963672013,1.3390691850674177,1.361720566377774,1.2895449036330602,1.342126487978718,1.4518638252653944,1.6633728850614524,1.5582813919379872,1.4686284837006685,1.42874847968582,1.085152993841838,1.1811843939986342,1.3089963739920012,1.5124924183670656,1.5124924183670656,1.3089963739920012,1.1811843939986342,1.085152993841838,1.42874847968582,1.4686284837006685,1.5582813919379872,1.6633728850614524,1.4518638252653944,1.342126487978718,1.2895449036330602,1.361720566377774,1.3390691850674177,2.1135601963672013,1.4874799944259034,1.7926898224178711,1.584388961931582,1.5867914113761434,1.735988854665956,1.5368561570693156,1.9360366404920484,1.4578214827166478,1.6701857137564855,2.1062263604341616,2.0075263733257755,1.5212893856644114,2.00990638700909,1.3831502596286656,2.2983513828834825,1.9053203907837477,4.280914528765986,3.344512943382261,2.497016468137817,5.341293038851984,2.177201992767261,4.361018630229035,6.387403956953162,5.048236693896485,2.595273725776378,7.516908722403525,25.893231464828432,14.028810047107587,8.14584129140285,15.568329716878544,23.78136989731783,21.91764509090947,32.13744783420577,23.27063818626752,24.55065196899352,52.86057975123696,31.4954245434392,49.2025937833613,12.626765084859084,38.296132245945,33.808681536867226,46.3652031959139,25.01970495571268,30.46551890972791,36.4284181692426,39.862175538162425,4.28840771540847,4.349501719306558"

            with col3:
                if st.button("🟢"):
                    st.session_state.pasted_data = "13.925,14.924,1.87152,10.78,3.1565239250796115,3.669636276254092,0.1211691859223678,0.6603649862081264,14.81,132.56,198.80622450046425,407.0805420038367,41.30326758504787,377.607150917067,329.6929704450757,233.94,148.7013816289654,85.91753652415252,69.78750390743814,73.20072885651093,73.1322894014404,98.02996518368482,34.041009405378574,51.28897759806096,37.05852623857877,42.71157200488174,4.121505249611095,63.62564340933166,31.534858757085843,26.046172390334792,31.967841793106,35.26869307443615,16.832909786511177,14.58838382535911,72.71765423745356,39.38033547082446,13.050879655426137,10.622103786120103,16.61259155569531,7.327163401325991,2.697414899833233,9.808442155880304,4.763893431803291,1.933088662754356,5.737719153982554,2.5647525565726794,2.712132672587128,1.7239571826365292,1.6810039195561426,1.2736319462766283,2.1208764527542963,1.431964899448816,1.5971458512336798,1.2528988180493683,0.9752943870497016,1.3979536027060762,1.1679119156365092,1.1380472712930905,1.0470923307776576,0.8319348985150075,1.05327854458485,1.023053339527435,1.0215374156567398,1.2524345353520694,1.1490328654912667,1.001824764196653,1.068374193811373,1.0746584124407246,1.130687521124672,0.93008616766653,1.0557527635382242,0.6299639484264786,0.5573043439644885,0.7358360777936935,1.1573668875401568,1.3547238103124235,1.0282593944166858,1.1624938941100904,1.1624938941100904,1.0282593944166858,1.3547238103124235,1.1573668875401568,0.7358360777936935,0.5573043439644885,0.6299639484264786,1.0557527635382242,0.93008616766653,1.130687521124672,1.0746584124407246,1.068374193811373,1.001824764196653,1.1490328654912667,1.2524345353520694,1.0215374156567398,1.023053339527435,1.05327854458485,0.8319348985150075,1.0470923307776576,1.1380472712930905,1.1679119156365092,1.3979536027060762,0.9752943870497016,1.2528988180493683,1.5971458512336798,1.431964899448816,2.1208764527542963,1.2736319462766283,1.6810039195561426,1.7239571826365292,2.712132672587128,2.5647525565726794,5.737719153982554,1.933088662754356,4.763893431803291,9.808442155880304,2.697414899833233,7.327163401325991,16.61259155569531,10.622103786120103,13.050879655426137,39.38033547082446,72.71765423745356,14.58838382535911,16.832909786511177,35.26869307443615,31.967841793106,26.046172390334792,31.534858757085843,63.62564340933166,4.121505249611095,42.71157200488174,37.05852623857877,51.28897759806096,34.041009405378574,98.02996518368482,73.1322894014404,73.20072885651093,69.78750390743814,85.91753652415252,148.7013816289654"

            with col4:
                if st.button("🔴"):
                    st.session_state.pasted_data = "17.895,18.892,0.5593599999999999,4.19,1.3399031272446529,1.451972451529298,0.0320338884493001,0.2706543858059262,6.720000000000001,76.88,140.49172965028924,242.71788485255345,14.893473125791424,92.5938578955408,158.00432313596815,69.92,17.648413440914005,17.60878433820803,15.942900926788193,41.39375918963057,31.65929981399516,28.51668016333828,11.837738400891023,44.59684556811312,23.26107343091225,31.31308591692754,26.198424253335173,15.122300481001137,11.947044300968017,35.65477050898056,40.74540871709828,3.170204831248526,17.743787859483145,7.832579625232223,14.202732782976067,11.234295213345131,25.86605384164161,10.297599158678702,15.900881342063846,10.847474399616363,1.2189751711523162,6.250109488078362,7.004108841778587,2.070034778184759,3.987326881330624,1.6221966296951873,1.8513863201267733,1.0579632953423768,0.6048262688858255,1.1045783463260963,0.2327169453534671,0.1513056928571692,0.5569245010654496,0.3110401781217685,0.5506589456930167,0.4080679794684767,0.1097516947495553,0.1624563203280561,0.3824027368042154,0.2869173274904243,0.497512987331642,0.323380254486268,0.2630683124703888,0.4477154213611007,0.4652196010429258,0.5571351111841539,0.0248571056015484,0.2282924971433172,0.2121943384031659,0.39426722460893,0.1437967244183503,0.3457206082414431,0.0580132488613787,0.1442489900632044,0.3821711343209582,0.2631918582884829,0.4013443380081823,0.3481501876481251,0.3481501876481251,0.4013443380081823,0.2631918582884829,0.3821711343209582,0.1442489900632044,0.0580132488613787,0.3457206082414431,0.1437967244183503,0.39426722460893,0.2121943384031659,0.2282924971433172,0.0248571056015484,0.5571351111841539,0.4652196010429258,0.4477154213611007,0.2630683124703888,0.323380254486268,0.497512987331642,0.2869173274904243,0.3824027368042154,0.1624563203280561,0.1097516947495553,0.4080679794684767,0.5506589456930167,0.3110401781217685,0.5569245010654496,0.1513056928571692,0.2327169453534671,1.1045783463260963,0.6048262688858255,1.0579632953423768,1.8513863201267733,1.6221966296951873,3.987326881330624,2.070034778184759,7.004108841778587,6.250109488078362,1.2189751711523162,10.847474399616363,15.900881342063846,10.297599158678702,25.86605384164161,11.234295213345131,14.202732782976067,7.832579625232223,17.743787859483145,3.170204831248526,40.74540871709828,35.65477050898056,11.947044300968017,15.122300481001137,26.198424253335173,31.31308591692754,23.26107343091225,44.59684556811312,11.837738400891023,28.51668016333828,31.65929981399516,41.39375918963057,15.942900926788193,17.60878433820803,17.648413440914005"
            # Create the text area below the buttons
            pasted_data = st.text_area("Paste a row of data (comma-separated values) to get predictions.", value=st.session_state.pasted_data, height=100)

            # Add Predict button
            predict_button = st.button("Predict")

            # Create a placeholder for prediction results
            prediction_result_placeholder = st.empty()

        # Create a container for visualization that will appear below the prediction result
        visualization_container = st.container()

        if pasted_data and predict_button:
            try:
                # Split the pasted data and create a DataFrame
                values = [float(x.strip()) for x in pasted_data.split(',')]
                feature_names = ['Start Timestamp', 'End Timestamp', 'Mean', 'Max', 'Standard Deviation', 'RMS',
                            'Kurtosis', 'Skewness', 'Peak-to-Peak', 'Abs Diff Signal', 'Alpha Power',
                            'Beta Power', 'Gamma Power', 'Delta Power', 'Theta Power'] + \
                            [f'FFT_{i}' for i in range(125)]

                if len(values) == len(feature_names):
                    input_data = dict(zip(feature_names, values))
                    input_df = pd.DataFrame([input_data])

                    # Make prediction
                    prediction = make_predictions(eeg_model, eeg_scaler, preprocess_eeg_data(input_df))

                    # Display prediction result - stacked as in the screenshot
                    with prediction_result_placeholder.container():
                        st.subheader("Prediction Result")
                        predicted_label = prediction[0]
                        command = label_to_command.get(predicted_label, "Unknown Command")

                        # Display results stacked (one below the other)
                        st.write(f"Predicted Label: {predicted_label}")

                        # Make command text slightly bigger using HTML
                        command_html = f"""
                        <div style="font-size: 1.2em; margin-top: 5px;">
                            <strong>Command:</strong> {command}
                        </div>
                        """
                        st.markdown(command_html, unsafe_allow_html=True)

                        st.markdown("---")  # Add a separator

                    # Create side-by-side visualization below the prediction result
                    with visualization_container:
                        # Use columns with specific width ratio (3:7) - 30% for Movement and 70% for Signal
                        viz_cols = st.columns([3, 7])

                        # Left column for wheelchair visualization - fixed size as in original but aligned left
                        with viz_cols[0]:
                            st.subheader("Movement Visualization")

                            # Define CSS for the animated wheelchair - keeping original size but aligned left
                            css = """
                            <style>
                            @keyframes moveLeft {
                                from { transform: translate(-50%, -50%); }
                                to { transform: translate(-150%, -50%); }
                            }
                            @keyframes moveRight {
                                from { transform: translate(-50%, -50%); }
                                to { transform: translate(50%, -50%); }
                            }
                            @keyframes moveForward {
                                from { transform: translate(-50%, -50%); }
                                to { transform: translate(-50%, -150%); }
                            }
                            @keyframes moveBackward {
                                from { transform: translate(-50%, -50%); }
                                to { transform: translate(-50%, 50%); }
                            }
                            .wheelchair-container {
                                position: relative;
                                width: 300px;
                                height: 300px;
                                margin: 0; /* Changed from margin: 0 auto to margin: 0 to align left */
                                border: 1px solid #f0f0f0;
                                border-radius: 10px;
                                background-color: #fafafa;
                                overflow: hidden;
                            }
                            .wheelchair {
                                position: absolute;
                                top: 50%;
                                left: 50%;
                                transform: translate(-50%, -50%);
                                font-size: 3em;
                                z-index: 10;
                                color: black;
                            }
                            .arrow {
                                position: absolute;
                                font-size: 0.9em;
                                z-index: 5;
                            }
                            .arrow-label {
                                font-size: 0.7em;
                            }
                            .forward {
                                top: 15%;
                                left: 50%;
                                transform: translateX(-50%);
                            }
                            .backward {
                                bottom: 15%;
                                left: 50%;
                                transform: translateX(-50%);
                            }
                            .left {
                                top: 50%;
                                left: 15%;
                                transform: translateY(-50%);
                            }
                            .right {
                                top: 50%;
                                right: 15%;
                                transform: translateY(-50%);
                            }
                            .move-left {
                                animation: moveLeft 2s forwards;
                            }
                            .move-right {
                                animation: moveRight 2s forwards;
                            }
                            .move-forward {
                                animation: moveForward 2s forwards;
                            }
                            .move-backward {
                                animation: moveBackward 2s forwards;
                            }
                            </style>
                            """

                            # Determine animation class based on prediction
                            animation_class = ""
                            if predicted_label == 0:  # Backward
                                animation_class = "move-backward"
                            elif predicted_label == 1:  # Left
                                animation_class = "move-left"
                            elif predicted_label == 2:  # Right
                                animation_class = "move-right"
                            elif predicted_label == 3:  # Forward
                                animation_class = "move-forward"

                            # Create HTML for the animated wheelchair
                            html = f"""
                            {css}
                            <div class="wheelchair-container">
                                <div class="arrow forward">⬆️<br><span class="arrow-label">Forward</span></div>
                                <div class="arrow backward">⬇️<br><span class="arrow-label">Backward</span></div>
                                <div class="arrow left">⬅️<br><span class="arrow-label">Left</span></div>
                                <div class="arrow right">➡️<br><span class="arrow-label">Right</span></div>
                                <div class="wheelchair {animation_class}">♿</div>
                            </div>
                            """

                            # Display the animated wheelchair
                            st.markdown(html, unsafe_allow_html=True)

                        # Right column for signal visualization - wider to show all values
                        with viz_cols[1]:
                            st.subheader("Signal Visualization")

                            # Create a plot showing all values as in the original
                            fig = go.Figure()

                            # Add main signal trace with all values
                            fig.add_trace(go.Scatter(
                                y=list(input_data.values())[2:],  # All values except timestamps
                                mode='lines+markers',
                                name='EEG Signal',
                                line=dict(color='#3498db', width=2),
                                marker=dict(size=5, color='#2c3e50')
                            ))

                            # Update layout for better appearance while keeping all data points
                            fig.update_layout(
                                title='Input EEG Signal',
                                xaxis_title='Feature',
                                yaxis_title='Value',
                                height=300,
                                margin=dict(l=10, r=10, t=40, b=80),
                                xaxis=dict(
                                    ticktext=list(input_data.keys())[2:],  # Skip timestamps
                                    tickvals=list(range(len(input_data)-2)),
                                    tickangle=45,
                                    tickfont=dict(size=8)
                                )
                            )

                            # Display the plot with left alignment
                            st.plotly_chart(fig, use_container_width=True)
                else:
                    st.error(f"Expected {len(feature_names)} values, got {len(values)}")
            except Exception as e:
                st.error(f"Error processing pasted data: {str(e)}")


    with tab3:
        # Emotion Batch Prediction content
        emotion_cols = st.columns([7, 3])

        with emotion_cols[0]:
            st.header("Emotion EEG Batch Prediction")
            st.write("Upload a CSV file containing EEG signals for emotion prediction.")

            # Add sample data download
            st.markdown("### Sample Data")
            try:
                sample_emotion = pd.read_csv('sample_emotion.csv')
                st.markdown(get_table_download_link(sample_emotion, 'sample_emotion.csv'), unsafe_allow_html=True)
            except Exception as e:
                st.error(f"Error loading sample data: {str(e)}")

            uploaded_file = st.file_uploader("Choose a CSV file for Emotion Detection", type="csv", key="emotion_uploader")

            if uploaded_file is not None:
                try:
                    df = pd.read_csv(uploaded_file)
                    st.write("Preview of uploaded data:")
                    st.dataframe(df.head())

                    X = preprocess_emotion_data(df)
                    if st.button("Make Emotion Predictions"):
                        with st.spinner("Processing emotions..."):
                            predictions = make_emotion_predictions(emotion_model, emotion_scaler, feature_selector, X)

                            if predictions is not None:
                                results_df = df.copy()
                                results_df['Predicted_Emotion'] = predictions
                                results_df['Emotion'] = results_df['Predicted_Emotion'].map(emotion_labels)

                                st.subheader("Emotion Prediction Results")
                                st.dataframe(results_df)

                                # Download predictions
                                csv = results_df.to_csv(index=False)
                                st.download_button(
                                    label="Download Emotion Predictions",
                                    data=csv,
                                    file_name='emotion_predictions.csv',
                                    mime='text/csv'
                                )

                                # Create emotion distribution visualization
                                emotion_counts = results_df['Predicted_Emotion'].value_counts().to_dict()

                                # Ensure all emotions are represented
                                for emotion in emotion_labels.keys():
                                    if emotion not in emotion_counts:
                                        emotion_counts[emotion] = 0

                                st.subheader("Emotion Distribution")
                                gauge_fig = create_emotion_gauge(emotion_counts)
                                st.plotly_chart(gauge_fig, use_container_width=True)

                                # Show emotion timeline
                                st.subheader("Emotion Timeline")
                                timeline_fig = px.line(
                                    results_df.reset_index(),
                                    x='index',
                                    y='Predicted_Emotion',
                                    title='Emotion Changes Over Time',
                                    labels={'index': 'Sample Number', 'Predicted_Emotion': 'Emotion'},
                                    color_discrete_sequence=[emotion_colors[1]]
                                )
                                timeline_fig.update_layout(yaxis=dict(ticktext=list(emotion_labels.values()),
                                                                    tickvals=list(emotion_labels.keys())))
                                st.plotly_chart(timeline_fig)
                            else:
                                st.error("Failed to make predictions. Please try again.")

                except Exception as e:
                    st.error(f"Error processing file: {str(e)}")

        # Right column for emotion information
        with emotion_cols[1]:
            st.markdown("""
            <h2 style="font-size: 1.8rem; font-weight: 600; margin-bottom: 0.5rem;">Emotion Categories</h2>
            """, unsafe_allow_html=True)

            for emotion_id, label in reversed(emotion_labels.items()):  # Reversed the order
                st.markdown(f"""
                <div class="emotion-box" style="background-color: {emotion_colors[emotion_id]}20; border: 2px solid {emotion_colors[emotion_id]}">
                    {label}
                </div>
                """, unsafe_allow_html=True)

            st.markdown("""
            ### About Emotion Detection
            The emotion detection model analyzes EEG signals to classify emotional states into three categories:
            - 😊 **Positive**: Happy, excited, or content states
            - 😐 **Neutral**: Calm or balanced emotional states
            - 😠 **Negative**: Stressed, frustrated, or upset states

            The model uses various EEG features including:
            - Power band ratios (Alpha, Beta, Gamma)
            - Statistical measures
            - Frequency domain features
            """)

    with tab4:
        st.header("Emotion Single Signal Prediction")

        # Create input container
        input_col = st.container()
        with input_col:
            st.write("Click the below buttons to get emotion predictions.")

            # Example data for each emotion (replace with your real samples)
            angry_sample = "13.3,30.4,-149.0,11.8,28.3,3.03,0.895,-4.52,1.72,-0.633,11.4,29.4,-158.0,11.7,27.8,12.1,30.6,-135.0,10.3,29.5,16.8,29.6,-150.0,11.8,28.8,12.8,32.1,-152.0,13.6,27.3,-0.667,-1.23,-23.8,1.32,-1.7,-5.37,-0.221,-8.63,-0.17,-0.987,-1.43,-2.78,-6.43,-1.91,0.508,-4.7,1.01,15.1,-1.49,0.716,-0.765,-1.55,17.3,-3.23,2.21,3.93,-2.56,2.2,-1.74,1.49,23.5,4.02,153.0,8.8,32.6,-1.69,-0.024,6.82,-0.0452,3.13,1020.0,20.1,1140000.0,62.6,-930.0,585000.0,849.0,1380000000.0,17400.0,4010000.0,3150000.0,4100.0,160000000000.0,46700.0,6930000.0,809000000.0,89800.0,111000000000000.0,5740000.0,22700000000.0,64.1,44.8,236.0,35.8,122.0,-5.5,-5.44,-82.6,4.9,-1.99,50.2,36.0,157.0,26.0,122.0,64.1,44.8,236.0,30.9,100.0,58.6,35.8,139.0,33.7,91.5,50.4,39.3,154.0,35.8,120.0,-13.9,-8.83,-79.8,-4.9,22.4,-8.41,0.159,17.6,-7.71,30.8,-0.155,-3.39,2.83,-9.8,1.99,5.5,8.99,97.4,-2.81,8.47,13.8,5.44,82.6,-4.9,-20.4,8.26,-3.55,-14.8,-2.09,-28.9,-32.4,20.7,-467.0,-8.89,-53.9,9.71,1.22,38.5,1.21,2.02,-25.6,22.6,-467.0,-5.08,-23.8,-32.4,20.7,-397.0,-8.89,-53.9,-22.7,21.9,-429.0,-2.9,-51.9,-22.1,25.4,-409.0,-7.68,-51.9,6.76,1.86,-70.4,3.81,30.1,-2.95,0.641,-38.5,-2.17,28.1,-3.47,-2.78,-57.9,2.6,28.1,-9.71,-1.22,31.9,-5.99,-2.03,-10.2,-4.64,12.5,-1.21,-2.02,-0.513,-3.42,-19.4,4.78,0.0071,4340.0,-1070.0,-38.1,788.0,954.0,77.5,380.0,-1070.0,-1450.0,-1360.0,-177.0,93.1,-1070.0,4170.0,-41.0,138.0,-1010.0,250.0,489.0,-75.4,10500.0,87.6,-90.8,39.9,-38.1,-41.0,8.1,27.0,-109.0,-113.0,-4.08,4.91,-217.0,246.0,-30.8,3.41,788.0,138.0,27.0,2030.0,82.8,-399.0,87.3,-492.0,-3000.0,1300.0,67.2,55.2,954.0,-1010.0,-109.0,82.8,3590.0,1760.0,-614.0,-106.0,-2910.0,-6310.0,691.0,-25.2,77.5,250.0,-113.0,-399.0,1760.0,3840.0,36.9,378.0,1040.0,-3910.0,356.0,-59.0,380.0,489.0,-4.08,87.3,-614.0,36.9,850.0,-280.0,2340.0,662.0,-138.0,-140.0,-1070.0,-75.4,4.91,-492.0,-106.0,378.0,-280.0,881.0,271.0,330.0,66.9,-57.5,-1450.0,10500.0,-217.0,-3000.0,-2910.0,1040.0,2340.0,271.0,36600.0,-1630.0,-257.0,-197.0,-1360.0,87.6,246.0,1300.0,-6310.0,-3910.0,662.0,330.0,-1630.0,14100.0,-1290.0,71.4,-177.0,-90.8,-30.8,67.2,691.0,356.0,-138.0,66.9,-257.0,-1290.0,683.0,5.79,93.1,39.9,3.41,55.2,-25.2,-59.0,-140.0,-57.5,-197.0,71.4,5.79,59.3,40500.0,18800.0,4880.0,2610.0,2430.0,745.0,637.0,405.0,182.0,24.3,2.49,-2.74e-13,8.06,6.01,-20.8,6.4,6.01,7.9,4.73,5.81,10.1,8.8,6.17,-5.13,-0.67,1.32,-0.978,0.761,0.313,0.47,-1.16,0.235,-0.11,-0.322,0.633,-0.188,1.52,0.655,-0.13,-0.763,0.476,0.625,0.248,0.0622,-0.0113,0.534,-0.521,-1.11,0.745,0.0482,-0.0197,0.0194,0.148,0.289,0.137,0.213,4.7,-0.0977,-0.187,-0.491,0.054,-1.92,-0.00337,-0.846,2.6,-0.71,-1.67,-0.000498,-3.92,0.17,-0.52,-0.823,0.592,0.152,0.645,-0.551,1.44,0.0782,0.377,-0.858,0.105,-0.199,-0.76,0.388,-0.194,-0.29,12.0,-0.279,-0.249,0.418,0.0,5.0,0.0,0.0,0.0,44000.0,-4080.0,43800.0,60600.0,-30.6,17800.0,110000.0,-20700.0,97800.0,61300.0,18500.0,20800.0,39300.0,3820.0,27200.0,56200.0,6160.0,34200.0,69600.0,6400.0,55200.0,122000.0,32800.0,52300.0,74400.0,-14300.0,28700.0,90400.0,7210.0,20600.0,52500.0,-7430.0,11100.0,40500.0,2650.0,29400.0,87900.0,-12600.0,10500.0,59300.0,-12000.0,47300.0,64700.0,6540.0,31500.0,39500.0,21100.0,9760.0,123000.0,9050.0,37000.0,92300.0,22300.0,24800.0,91900.0,8570.0,14500.0,122000.0,14300.0,7470.0,61600.0,8260.0,48800.0,66000.0,5760.0,20000.0,53200.0,2620.0,36500.0,118000.0,-12400.0,24700.0,78300.0,-2500.0,14000.0,-139.0,233.0,-125.0,-125.0,233.0,173.0,-32.7,43.7,43.7,-32.7,-143.0,136.0,-110.0,-110.0,136.0,-93.4,179.0,-81.5,-81.5,179.0,125.0,5.9,22.0,22.0,5.9,-170.0,166.0,-116.0,-116.0,166.0,-410.0,402.0,-151.0,-151.0,402.0,-236.0,288.0,-74.5,-74.5,288.0,-309.0,252.0,-143.0,-143.0,252.0,-86.3,170.0,-85.9,-85.9,170.0,207.0,-56.7,30.7,30.7,-56.7,-95.2,114.0,-109.0,-109.0,114.0,-112.0,189.0,-76.4,-76.4,189.0,119.0,46.8,-23.3,-23.3,46.8,-125.0,155.0,-113.0,-113.0,155.0,-127.0,173.0,-52.9,-52.9,173.0,137.0,46.3,10.7,10.7,46.3,-227.0,186.0,-93.3,-93.3,186.0,-108.0,181.0,-75.5,-75.5,181.0,188.0,4.13,-8.69,-8.69,4.13,-355.0,289.0,-160.0,-160.0,289.0,-366.0,318.0,-107.0,-107.0,318.0,27.0,98.0,-30.6,-30.6,98.0,-363.0,265.0,-133.0,-133.0,265.0,-240.0,321.0,-182.0,-182.0,321.0,66.2,185.0,-128.0,-128.0,185.0,-68.1,83.9,-114.0,-114.0,83.9,-300.0,318.0,-145.0,-145.0,318.0,143.0,28.2,-18.0,-18.0,28.2,5.54,59.4,-109.0,-109.0,59.4,-6.22,154.0,-101.0,-101.0,154.0,252.0,-95.8,42.2,42.2,-95.8,-55.2,81.3,-104.0,-104.0,81.3,-33.7,162.0,-113.0,-113.0,162.0,81.8,44.6,-17.1,-17.1,44.6,-134.0,134.0,-118.0,-118.0,134.0,-117.0,187.0,-97.8,-97.8,187.0,232.0,-40.4,42.9,42.9,-40.4,-9.44,83.2,-105.0,-105.0,83.2,-92.4,202.0,-111.0,-111.0,202.0,-3.1,120.0,3.97,3.97,120.0,-215.0,191.0,-139.0,-139.0,191.0,-192.0,288.0,-160.0,-160.0,288.0,41.7,68.7,-16.0,-16.0,68.7,-54.2,65.9,-109.0,-109.0,65.9,-64.7,131.0,-89.6,-89.6,131.0,409.0,-117.0,-1.92,-1.92,-117.0,18.3,79.9,-139.0,-139.0,79.9,-327.0,335.0,-156.0,-156.0,335.0,-56.4,112.0,-11.1,-11.1,112.0,-49.0,95.8,-152.0,-152.0,95.8,-171.0,203.0,-124.0,-124.0,203.0,398.0,-132.0,-1.2,-1.2,-132.0,-180.0,136.0,-108.0,-108.0,136.0,-287.0,214.0,-49.6,-49.6,214.0,-52.8,142.0,-12.2,-12.2,142.0,-132.0,135.0,-87.9,-87.9,135.0,-249.0,286.0,-99.5,-99.5,286.0,74.8,124.0,-5.97,-5.97,124.0,-81.8,184.0,-114.0,-114.0,184.0,-170.0,219.0,-39.3,-39.3,219.0,200.0,30.0,30.3,30.3,30.0,-221.0,239.0,-143.0,-143.0,239.0,-261.0,283.0,-142.0,-142.0,283.0,88.7,78.7,-31.7,-31.7,78.7,-27.3,79.6,-97.3,-97.3,79.6,-82.2,166.0,-95.9,-95.9,166.0,187.0,21.3,18.5,18.5,21.3,-313.0,215.0,-87.9,-87.9,215.0,-337.0,345.0,-186.0,-186.0,345.0,-9.13,127.0,-46.7,-46.7,127.0,-68.5,68.3,-99.0,-99.0,68.3,-124.0,186.0,-115.0,-115.0,186.0,266.0,-85.5,39.1,39.1,-85.5,-72.3,111.0,-109.0,-109.0,111.0,-89.7,150.0,-101.0,-101.0,150.0,36.4,104.0,-36.4,-36.4,104.0,-164.0,168.0,-142.0,-142.0,168.0,-211.0,273.0,-141.0,-141.0,273.0,18.4,111.0,-21.2,-21.2,111.0,31.3,80.2,-102.0,-102.0,80.2,-144.0,222.0,-89.5,-89.5,222.0,167.0,-20.3,67.0,67.0,-20.3,-254.0,246.0,-146.0,-146.0,246.0,-302.0,300.0,-104.0,-104.0,300.0,273.0,-47.3,23.9,23.9,-47.3,-102.0,113.0,-71.1,-71.1,113.0,-50.2,193.0,-125.0,-125.0,193.0,259.0,-34.8,23.1,23.1,-34.8,-29.5,119.0,-101.0,-101.0,119.0,-131.0,289.0,-153.0,-153.0,289.0,242.0,-7.16,32.6,32.6,-7.16,-48.6,126.0,-86.9,-86.9,126.0,-258.0,314.0,-126.0,-126.0,314.0,169.0,0.19,36.4,36.4,0.19,-99.9,122.0,-98.1,-98.1,122.0,-271.0,281.0,-117.0,-117.0,281.0,-274.0,353.0,-117.0,-117.0,353.0,-89.6,128.0,-109.0,-109.0,128.0,-104.0,196.0,-106.0,-106.0,196.0,250.0,-69.3,42.2,42.2,-69.3,-234.0,184.0,-107.0,-107.0,184.0,-161.0,214.0,-101.0,-101.0,214.0,260.0,-57.0,34.2,34.2,-57.0,-269.0,211.0,-102.0,-102.0,211.0,-356.0,328.0,-117.0,-117.0,328.0,41.9,91.9,-8.89,-8.89,91.9,27.4,70.1,-113.0,-113.0,70.1,-298.0,232.0,-55.4,-55.4,232.0,302.0,-43.6,6.83,6.83,-43.6,-66.2,130.0,-117.0,-117.0,130.0,-398.0,349.0,-156.0,-156.0,349.0,-78.6,153.0,-23.0,-23.0,153.0,-22.9,53.8,-91.5,-91.5,53.8,-300.0,232.0,-61.3,-61.3,232.0,282.0,-69.7,42.8,42.8,-69.7,-182.0,120.0,-84.5,-84.5,120.0,-141.0,174.0,-90.9,-90.9,174.0,42.3,99.1,-8.69,-8.69,99.1,-240.0,226.0,-125.0,-125.0,226.0,-168.0,232.0,-115.0,-115.0,232.0,221.0,8.6,-16.7,-16.7,8.6,-161.0,92.2,-41.0,-41.0,92.2,-205.0,272.0,-159.0,-159.0,272.0,60.4,87.5,-2.16,-2.16,87.5,-340.0,243.0,-96.4,-96.4,243.0,-337.0,336.0,-139.0,-139.0,336.0,230.0,-9.54,9.43,9.43,-9.54,-78.0,78.7,-60.3,-60.3,78.7,-294.0,288.0,-120.0,-120.0,288.0,81.8,80.8,-5.45,-5.45,80.8,0.00662,68.6,-87.4,-87.4,68.6,-266.0,333.0,-180.0,-180.0,333.0,-73.8,143.0,-13.0,-13.0,143.0,153.0,21.9,-137.0,-137.0,21.9,-139.0,250.0,-174.0,-174.0,250.0,197.0,-48.3,33.8,33.8,-48.3,-116.0,127.0,-91.9,-91.9,127.0,-100.0,173.0,-119.0,-119.0,173.0,173.0,47.8,-56.6,-56.6,47.8,-110.0,144.0,-135.0,-135.0,144.0,-241.0,280.0,-154.0,-154.0,280.0,64.0,123.0,-57.3,-57.3,123.0,-83.4,104.0,-87.4,-87.4,104.0,-169.0,276.0,-164.0,-164.0,276.0,276.0,-74.4,28.0,28.0,-74.4,-261.0,227.0,-108.0,-108.0,227.0,-275.0,351.0,-196.0,-196.0,351.0,95.3,110.0,-31.5,-31.5,110.0,-9.4,89.3,-47.0,-47.0,89.3,15.8,29.6,-149.0,15.7,29.0,2.13,-2.63,4.53,6.18,2.01,16.7,29.7,-150.0,11.6,28.7,12.9,32.2,-152.0,13.6,27.3,19.5,27.3,-146.0,24.1,31.0,14.3,29.3,-147.0,13.6,29.1,3.83,-2.5,2.17,-1.94,1.45,-2.83,2.38,-4.02,-12.5,-2.23,2.39,0.379,-2.86,-1.96,-0.349,-6.67,4.88,-6.19,-10.6,-3.67,-1.44,2.88,-5.04,-0.0125,-1.8,5.23,-2.0,1.15,10.5,1.88,22.7,4.82,158.0,11.0,33.5,0.0911,1.21,3.2,3.17,-0.714,3080.0,-37.0,209000.0,620.0,3180.0,464000.0,1720.0,1410000000.0,45600.0,3770000.0,5150000.0,-10200.0,8280000000.0,623000.0,22000000.0,525000000.0,226000.0,102000000000000.0,28100000.0,19300000000.0,63.6,39.7,151.0,49.0,122.0,4.91,-1.78,1.99,13.3,-12.0,58.7,35.7,141.0,33.5,94.3,49.5,39.7,149.0,35.7,122.0,63.6,37.5,126.0,47.6,110.0,51.5,37.9,151.0,49.0,106.0,9.17,-3.98,-7.7,-2.2,-27.3,-4.91,-1.77,15.8,-14.1,-15.3,7.18,-2.2,-9.7,-15.5,-11.5,-14.1,2.22,23.5,-11.9,12.0,-1.99,1.78,-1.99,-13.3,15.8,12.1,-0.432,-25.5,-1.42,3.75,-23.9,13.6,-507.0,-7.53,-54.3,6.01,-8.28,-76.1,1.86,-3.9,-23.9,21.9,-431.0,-2.64,-50.0,-22.4,25.4,-408.0,-7.53,-50.4,-17.9,17.9,-507.0,-5.67,-20.6,-15.6,13.6,-481.0,-4.66,-54.3,-1.55,-3.47,-23.3,4.89,0.411,-6.01,3.98,76.1,3.03,-29.4,-8.27,8.28,49.8,2.02,4.31,-4.45,7.45,99.4,-1.86,-29.9,-6.72,11.7,73.2,-2.87,3.9,-2.27,4.29,-26.2,-1.0,33.8,4580.0,-1120.0,-118.0,768.0,970.0,128.0,-6.21,433.0,-1540.0,-1200.0,631.0,309.0,-1120.0,4060.0,-53.5,157.0,-1090.0,520.0,236.0,-206.0,9150.0,-325.0,246.0,252.0,-118.0,-53.5,17.8,5.69,15.9,-9.71,-6.09,-33.1,-284.0,84.4,-6.73,-85.6,768.0,157.0,5.69,1900.0,561.0,-894.0,63.9,-205.0,-4050.0,1130.0,-49.1,-72.4,970.0,-1090.0,15.9,561.0,1640.0,24.0,-101.0,-144.0,-3810.0,-3480.0,-571.0,-104.0,128.0,520.0,-9.71,-894.0,24.0,2770.0,-116.0,-35.6,1880.0,-3350.0,361.0,-360.0,-6.21,236.0,-6.09,63.9,-101.0,-116.0,163.0,56.2,1190.0,17.8,9.26,-20.9,433.0,-206.0,-33.1,-205.0,-144.0,-35.6,56.2,208.0,771.0,455.0,238.0,155.0,-1540.0,9150.0,-284.0,-4050.0,-3810.0,1880.0,1190.0,771.0,39900.0,-2760.0,1440.0,968.0,-1200.0,-325.0,84.4,1130.0,-3480.0,-3350.0,17.8,455.0,-2760.0,18800.0,2020.0,-280.0,631.0,246.0,-6.73,-49.1,-571.0,361.0,9.26,238.0,1440.0,2020.0,1410.0,-53.9,309.0,252.0,-85.6,-72.4,-104.0,-360.0,-20.9,155.0,968.0,-280.0,-53.9,731.0,43400.0,20300.0,4970.0,3060.0,2460.0,794.0,646.0,209.0,240.0,45.3,9.47,8.23e-13,8.16,6.84,-18.8,5.08,5.54,7.38,0.207,0.0867,10.3,9.53,6.7,6.13,-0.491,1.98,-5.08,1.22,-0.286,-0.00367,4.42,-0.139,-0.117,0.372,-0.0491,0.633,1.65,2.76,-0.628,-1.53,-0.0357,0.689,0.569,0.222,-0.183,0.688,-0.635,-0.553,2.93,1.19,-0.0396,-0.478,0.834,0.0157,0.192,0.366,8.77,-2.3,-0.213,-0.552,-0.696,0.688,0.112,-1.17,-8.61,-0.837,-0.972,0.47,-0.364,-0.446,0.203,-0.94,0.411,-0.277,-0.529,0.694,1.01,0.823,0.408,-0.311,0.0554,-0.113,1.45,-0.166,-0.165,-0.148,-0.638,0.294,0.284,0.211,0.0,5.0,0.0,0.0,0.0,40300.0,-4550.0,37900.0,118000.0,13800.0,7160.0,80300.0,14200.0,43300.0,77400.0,658.0,40400.0,52800.0,9080.0,15700.0,81500.0,6820.0,19800.0,181000.0,854.0,10200.0,68000.0,-43500.0,13800.0,84200.0,-18100.0,21100.0,77700.0,20000.0,40300.0,108000.0,3220.0,18100.0,93700.0,13100.0,16400.0,150000.0,12200.0,17800.0,137000.0,-4070.0,18600.0,53400.0,-6150.0,41500.0,91400.0,7300.0,13400.0,103000.0,-711.0,33200.0,158000.0,-2610.0,33500.0,143000.0,6820.0,5420.0,83400.0,-14600.0,-6100.0,134000.0,-11400.0,22300.0,50100.0,4540.0,20000.0,98000.0,6050.0,4590.0,94700.0,-2640.0,-2990.0,90700.0,1630.0,18000.0,-110.0,159.0,-109.0,-109.0,159.0,53.1,99.1,-34.9,-34.9,99.1,-179.0,172.0,-142.0,-142.0,172.0,-198.0,269.0,-141.0,-141.0,269.0,6.48,114.0,-20.6,-20.6,114.0,42.4,76.7,-103.0,-103.0,76.7,-155.0,225.0,-88.7,-88.7,225.0,177.0,-23.4,66.0,66.0,-23.4,-263.0,249.0,-145.0,-145.0,249.0,-293.0,298.0,-105.0,-105.0,298.0,265.0,-44.7,24.9,24.9,-44.7,-94.5,110.0,-72.2,-72.2,110.0,-57.6,196.0,-124.0,-124.0,196.0,266.0,-37.1,22.0,22.0,-37.1,-36.3,121.0,-99.9,-99.9,121.0,-125.0,287.0,-154.0,-154.0,287.0,236.0,-5.09,33.8,33.8,-5.09,-42.5,124.0,-88.0,-88.0,124.0,-264.0,316.0,-125.0,-125.0,316.0,174.0,-1.68,35.3,35.3,-1.68,-105.0,124.0,-97.0,-97.0,124.0,-265.0,279.0,-118.0,-118.0,279.0,-279.0,355.0,-116.0,-116.0,355.0,-84.5,127.0,-111.0,-111.0,127.0,-109.0,198.0,-105.0,-105.0,198.0,255.0,-70.9,41.1,41.1,-70.9,-239.0,185.0,-106.0,-106.0,185.0,-157.0,212.0,-102.0,-102.0,212.0,255.0,-55.5,35.3,35.3,-55.5,-264.0,210.0,-103.0,-103.0,210.0,-360.0,329.0,-116.0,-116.0,329.0,46.0,90.5,-9.91,-9.91,90.5,23.5,71.5,-112.0,-112.0,71.5,-294.0,230.0,-56.4,-56.4,230.0,298.0,-42.2,7.82,7.82,-42.2,-62.4,129.0,-118.0,-118.0,129.0,-401.0,350.0,-155.0,-155.0,350.0,-75.0,152.0,-24.0,-24.0,152.0,-26.4,55.2,-90.6,-90.6,55.2,-297.0,231.0,-62.2,-62.2,231.0,278.0,-68.3,43.7,43.7,-68.3,-178.0,119.0,-85.3,-85.3,119.0,-144.0,175.0,-90.0,-90.0,175.0,45.5,97.7,-9.52,-9.52,97.7,-244.0,227.0,-124.0,-124.0,227.0,-165.0,230.0,-116.0,-116.0,230.0,218.0,9.98,-15.9,-15.9,9.98,-158.0,90.8,-41.8,-41.8,90.8,-208.0,274.0,-158.0,-158.0,274.0,63.5,86.1,-2.87,-2.87,86.1,-343.0,244.0,-95.7,-95.7,244.0,-334.0,335.0,-140.0,-140.0,335.0,227.0,-8.02,10.1,10.1,-8.02,-75.0,77.2,-60.9,-60.9,77.2,-297.0,290.0,-119.0,-119.0,290.0,84.9,79.2,-5.99,-5.99,79.2,-3.03,70.3,-86.9,-86.9,70.3,-263.0,331.0,-181.0,-181.0,331.0,-76.9,145.0,-12.6,-12.6,145.0,156.0,20.0,-137.0,-137.0,20.0,-142.0,252.0,-173.0,-173.0,252.0,200.0,-50.3,33.5,33.5,-50.3,-119.0,129.0,-91.6,-91.6,129.0,-96.6,171.0,-119.0,-119.0,171.0,169.0,50.2,-56.5,-56.5,50.2,-107.0,141.0,-136.0,-136.0,141.0,-245.0,282.0,-154.0,-154.0,282.0,68.1,120.0,-57.2,-57.2,120.0,-87.7,107.0,-87.6,-87.6,107.0,-164.0,273.0,-164.0,-164.0,273.0,271.0,-70.5,27.7,27.7,-70.5,-256.0,223.0,-108.0,-108.0,223.0,-281.0,356.0,-197.0,-197.0,356.0,102.0,104.0,-31.2,-31.2,104.0,-16.1,96.2,-46.9,-46.9,96.2,-168.0,200.0,-121.0,-121.0,200.0,226.0,-45.0,44.0,44.0,-45.0,-44.2,129.0,-99.7,-99.7,129.0,-295.0,332.0,-142.0,-142.0,332.0,41.5,130.0,-11.3,-11.3,130.0,89.1,32.7,-54.7,-54.7,32.7,-255.0,277.0,-64.7,-64.7,277.0,262.0,-50.9,78.9,78.9,-50.9,-43.6,95.6,-67.4,-67.4,95.6,-110.0,188.0,-89.3,-89.3,188.0,99.6,65.7,7.21,7.21,65.7,-245.0,226.0,-100.0,-100.0,226.0,-171.0,236.0,-104.0,-104.0,236.0,164.0,-12.0,56.0,56.0,-12.0,-56.6,72.6,-36.7,-36.7,72.6,-92.1,239.0,-140.0,-140.0,239.0,181.0,69.3,-19.0,-19.0,69.3,21.2,109.0,-96.0,-96.0,109.0,-409.0,422.0,-192.0,-192.0,422.0,127.0,101.0,-48.2,-48.2,101.0,69.0,54.3,-85.2,-85.2,54.3,-156.0,185.0,-74.3,-74.3,185.0,272.0,-53.5,32.9,32.9,-53.5,-71.2,56.4,-48.7,-48.7,56.4,-369.0,342.0,-146.0,-146.0,342.0,-53.1,150.0,-2.49,-2.49,150.0,-40.1,51.4,-63.4,-63.4,51.4,-251.0,267.0,-120.0,-120.0,267.0,292.0,-42.5,7.23,7.23,-42.5,-87.0,106.0,-64.6,-64.6,106.0,-209.0,229.0,-116.0,-116.0,229.0,147.0,44.0,-14.0,-14.0,44.0,-173.0,163.0,-121.0,-121.0,163.0,-281.0,270.0,-118.0,-118.0,270.0,190.0,2.95,4.7,4.7,2.95,-40.4,65.1,-76.0,-76.0,65.1,-192.0,329.0,-209.0,-209.0,329.0,67.8,92.2,-8.46,-8.46,92.2,-194.0,167.0,-81.8,-81.8,167.0,-384.0,396.0,-210.0,-210.0,396.0,90.3,99.7,-35.3,-35.3,99.7,-57.5,56.8,-48.8,-48.8,56.8,-192.0,216.0,-107.0,-107.0,216.0,297.0,-77.3,34.5,34.5,-77.3,-66.8,106.0,-111.0,-111.0,106.0,-276.0,290.0,-156.0,-156.0,290.0,153.0,51.7,-23.5,-23.5,51.7,-15.3,75.9,-73.2,-73.2,75.9,-306.0,277.0,-89.5,-89.5,277.0,251.0,-11.7,14.8,14.8,-11.7,-48.4,76.4,-72.1,-72.1,76.4,-385.0,306.0,-120.0,-120.0,306.0,-6.31,121.0,-32.6,-32.6,121.0,-217.0,191.0,-114.0,-114.0,191.0,-364.0,333.0,-162.0,-162.0,333.0,45.8,102.0,-39.6,-39.6,102.0,73.5,27.9,-104.0,-104.0,27.9,-60.0,212.0,-159.0,-159.0,212.0,222.0,-49.3,34.4,34.4,-49.3,-270.0,172.0,-55.6,-55.6,172.0,-450.0,396.0,-194.0,-194.0,396.0,-28.3,166.0,-48.6,-48.6,166.0,-94.0,97.8,-74.7,-74.7,97.8,-226.0,220.0,-94.0,-94.0,220.0,212.0,-39.7,47.7,47.7,-39.7,-86.4,84.7,-47.9,-47.9,84.7,-272.0,267.0,-93.0,-93.0,267.0,113.0,65.3,1.65,1.65,65.3,83.5,31.5,-75.6,-75.6,31.5,-186.0,293.0,-154.0,-154.0,293.0,133.0,125.0,-71.9,-71.9,125.0,162.0,-2.6,-32.8,-32.8,-2.6,-130.0,215.0,-115.0,-115.0,215.0,151.0,6.11,20.0,20.0,6.11,-53.0,176.0,-167.0,-167.0,176.0"
            neutral_sample = "13.4,31.1,162.0,19.8,23.8,4.87,3.03,457.0,0.777,-4.06,10.3,29.4,-59.1,19.5,23.0,11.6,29.8,-74.1,19.3,28.5,10.2,29.5,298.0,19.8,19.5,21.4,35.7,480.0,20.5,23.9,-1.37,-0.385,15.0,0.172,-5.52,0.127,-0.0969,-357.0,-0.355,3.45,-11.1,-6.27,-539.0,-1.02,-0.862,1.49,0.288,-372.0,-0.527,8.97,-9.74,-5.89,-554.0,-1.19,4.66,-11.2,-6.17,-182.0,-0.661,-4.32,27.7,6.96,416.0,13.7,30.5,12.2,6.19,-12.8,8.87,4.09,-9200.0,98.6,-1890000.0,-3160.0,-1940.0,1850000.0,10600.0,70300000000.0,226000.0,2370000.0,-74100000.0,9920.0,-265000000000.0,-9130000.0,-10200000.0,8980000000.0,3140000.0,3.6e+16,513000000.0,8710000000.0,70.0,51.5,1050.0,51.4,100.0,19.2,15.4,296.0,13.1,12.7,44.9,36.1,758.0,38.4,87.4,50.8,35.5,604.0,34.9,83.1,40.3,39.8,955.0,36.7,88.7,70.0,51.5,1050.0,51.4,100.0,-5.82,0.655,155.0,3.47,4.26,4.66,-3.7,-197.0,1.62,-1.36,-25.1,-15.4,-296.0,-13.1,-12.7,10.5,-4.35,-352.0,-1.85,-5.61,-19.2,-16.1,-451.0,-16.6,-17.0,-29.7,-11.7,-99.1,-14.7,-11.4,-78.2,9.86,-685.0,-38.1,-58.6,-47.9,-13.9,230.0,-39.9,-13.0,-25.5,23.7,-685.0,1.83,-35.8,-30.3,24.6,-684.0,3.89,-45.7,-31.2,25.6,-455.0,3.4,-58.6,-78.2,9.86,-0.529,-38.1,-27.7,4.82,-0.921,-1.33,-2.06,9.9,5.74,-1.91,-230.0,-1.57,22.9,52.7,13.9,-685.0,39.9,-8.08,0.922,-0.992,-228.0,0.49,13.0,47.9,14.8,-683.0,42.0,-18.0,47.0,15.8,-455.0,41.5,-30.9,1230.0,203.0,-1030.0,-1490.0,1890.0,1920.0,-1150.0,3290.0,-125.0,-286.0,-2590.0,-2570.0,203.0,22800.0,5810.0,-545.0,-7420.0,-4730.0,3490.0,1850.0,-35700.0,-1770.0,6400.0,6720.0,-1030.0,5810.0,31700.0,-2390.0,-51800.0,4800.0,-4210.0,-3440.0,-10500.0,-1130.0,-6830.0,-7890.0,-1490.0,-545.0,-2390.0,93200.0,7970.0,-43800.0,-682.0,-886.0,-32200.0,3060.0,40900.0,1960.0,1890.0,-7420.0,-51800.0,7970.0,85200.0,-9030.0,7950.0,6440.0,12000.0,1710.0,12400.0,14800.0,1920.0,-4730.0,4800.0,-43800.0,-9030.0,96900.0,6150.0,1850.0,14200.0,-6120.0,-17900.0,11000.0,-1150.0,3490.0,-4210.0,-682.0,7950.0,6150.0,15500.0,-1900.0,-10900.0,132.0,-6070.0,27600.0,3290.0,1850.0,-3440.0,-886.0,6440.0,1850.0,-1900.0,16100.0,-3940.0,-979.0,-5200.0,-6210.0,-125.0,-35700.0,-10500.0,-32200.0,12000.0,14200.0,-10900.0,-3940.0,77100.0,2280.0,-15900.0,-21300.0,-286.0,-1770.0,-1130.0,3060.0,1710.0,-6120.0,132.0,-979.0,2280.0,856.0,-373.0,1000.0,-2590.0,6400.0,-6830.0,40900.0,12400.0,-17900.0,-6070.0,-5200.0,-15900.0,-373.0,44700.0,-9080.0,-2570.0,6720.0,-7890.0,1960.0,14800.0,11000.0,27600.0,-6210.0,-21300.0,1000.0,-9080.0,55100.0,173000.0,128000.0,103000.0,57700.0,45100.0,23900.0,5910.0,1400.0,981.0,206.0,57.4,4.63e-12,4.52,7.71,0.562,10.6,7.81,11.3,7.25,9.35,10.1,-15.4,9.65,9.85,0.975,1.91,1.34,0.738,-0.356,-0.0623,-0.0647,-0.194,2.51,-2.92,-0.129,-2.04,-1.12,-6.39,-0.43,-0.738,0.081,0.0995,-0.543,0.176,2.38,0.669,0.881,-0.351,0.372,0.16,0.175,-3.78,-0.477,-0.652,-0.828,-0.082,-1.73,0.144,0.802,-1.25,-0.722,-0.378,0.00741,0.507,-0.292,-0.905,-5.87,-0.113,1.89,-1.03,0.18,0.902,2.55,-0.459,0.33,0.405,-2.0,-10.7,1.18,0.86,0.398,3.25,-1.58,-0.102,-4.45,0.979,0.876,-1.14,-0.202,0.107,0.0,4.99,0.0,0.0,0.0,279000.0,96800.0,-10900.0,148000.0,-6190.0,-31500.0,210000.0,58900.0,11500.0,134000.0,-19100.0,64800.0,9710.0,-137000.0,143000.0,55000.0,-13700.0,203000.0,-32200.0,-178000.0,272000.0,-84800.0,-32600.0,424000.0,-11900.0,91700.0,356000.0,-16700.0,134000.0,561000.0,1150.0,82200.0,571000.0,-21700.0,96800.0,619000.0,10300.0,136000.0,642000.0,7020.0,1660.0,380000.0,-19500.0,14200.0,454000.0,-53300.0,-16500.0,171000.0,-198000.0,-230000.0,182000.0,-46300.0,-52500.0,171000.0,-118000.0,-154000.0,223000.0,-56900.0,-113000.0,148000.0,-30900.0,-86200.0,109000.0,-25200.0,-44600.0,161000.0,-18400.0,-40400.0,99400.0,-129000.0,-188000.0,165000.0,2510.0,-5790.0,138000.0,-516.0,497.0,-288.0,-288.0,497.0,-496.0,502.0,-238.0,-238.0,502.0,18.3,50.2,20.7,20.7,50.2,-494.0,438.0,-212.0,-212.0,438.0,-295.0,349.0,-145.0,-145.0,349.0,8.7,129.0,-53.1,-53.1,129.0,-601.0,544.0,-282.0,-282.0,544.0,-441.0,403.0,-176.0,-176.0,403.0,92.3,-10.6,30.2,30.2,-10.6,-406.0,373.0,-234.0,-234.0,373.0,-224.0,262.0,-136.0,-136.0,262.0,222.0,-71.0,20.7,20.7,-71.0,-622.0,538.0,-272.0,-272.0,538.0,-403.0,430.0,-206.0,-206.0,430.0,413.0,-185.0,79.7,79.7,-185.0,-356.0,314.0,-138.0,-138.0,314.0,-12.1,100.0,-52.1,-52.1,100.0,472.0,-218.0,51.8,51.8,-218.0,-103.0,138.0,-110.0,-110.0,138.0,-286.0,260.0,-65.2,-65.2,260.0,471.0,-241.0,118.0,118.0,-241.0,-311.0,251.0,-118.0,-118.0,251.0,-34.2,110.0,-57.2,-57.2,110.0,557.0,-374.0,174.0,174.0,-374.0,-5.51,51.6,-103.0,-103.0,51.6,363.0,-200.0,44.4,44.4,-200.0,495.0,-288.0,128.0,128.0,-288.0,-1.07,22.6,-48.4,-48.4,22.6,326.0,-190.0,49.2,49.2,-190.0,894.0,-573.0,197.0,197.0,-573.0,64.7,-1.84,-54.8,-54.8,-1.84,257.0,-106.0,43.3,43.3,-106.0,671.0,-454.0,216.0,216.0,-454.0,-34.5,60.8,-63.5,-63.5,60.8,221.0,-130.0,62.6,62.6,-130.0,729.0,-540.0,239.0,239.0,-540.0,220.0,-75.2,-66.6,-66.6,-75.2,333.0,-182.0,54.9,54.9,-182.0,706.0,-458.0,196.0,196.0,-458.0,25.1,-42.1,-25.9,-25.9,-42.1,77.3,28.0,-24.3,-24.3,28.0,506.0,-284.0,108.0,108.0,-284.0,19.2,59.7,-82.4,-82.4,59.7,178.0,-1.45,-31.9,-31.9,-1.45,701.0,-346.0,123.0,123.0,-346.0,-123.0,147.0,-104.0,-104.0,147.0,74.6,69.0,-50.0,-50.0,69.0,250.0,-121.0,51.4,51.4,-121.0,-605.0,487.0,-260.0,-260.0,487.0,-490.0,443.0,-183.0,-183.0,443.0,349.0,-200.0,106.0,106.0,-200.0,-195.0,183.0,-135.0,-135.0,183.0,-70.0,125.0,-17.1,-17.1,125.0,234.0,-132.0,104.0,104.0,-132.0,-466.0,333.0,-134.0,-134.0,333.0,-164.0,227.0,-129.0,-129.0,227.0,420.0,-200.0,58.8,58.8,-200.0,-451.0,386.0,-221.0,-221.0,386.0,-143.0,218.0,-111.0,-111.0,218.0,272.0,-114.0,61.0,61.0,-114.0,-327.0,305.0,-185.0,-185.0,305.0,-188.0,285.0,-146.0,-146.0,285.0,263.0,-67.9,7.63,7.63,-67.9,-323.0,313.0,-187.0,-187.0,313.0,-94.4,177.0,-76.5,-76.5,177.0,341.0,-137.0,62.5,62.5,-137.0,-277.0,296.0,-208.0,-208.0,296.0,-85.5,165.0,-67.9,-67.9,165.0,293.0,-82.8,24.3,24.3,-82.8,-640.0,531.0,-258.0,-258.0,531.0,-434.0,444.0,-212.0,-212.0,444.0,464.0,-217.0,40.7,40.7,-217.0,-185.0,215.0,-153.0,-153.0,215.0,73.3,56.3,-39.0,-39.0,56.3,258.0,-96.5,49.8,49.8,-96.5,-394.0,359.0,-225.0,-225.0,359.0,-55.6,158.0,-87.1,-87.1,158.0,501.0,-261.0,78.2,78.2,-261.0,-230.0,181.0,-144.0,-144.0,181.0,178.0,25.1,-58.5,-58.5,25.1,391.0,-237.0,108.0,108.0,-237.0,-313.0,220.0,-90.6,-90.6,220.0,25.8,122.0,-79.0,-79.0,122.0,297.0,-168.0,102.0,102.0,-168.0,-214.0,206.0,-150.0,-150.0,206.0,211.0,-18.8,-53.4,-53.4,-18.8,606.0,-436.0,201.0,201.0,-436.0,11.9,-23.0,-41.9,-41.9,-23.0,324.0,-200.0,76.0,76.0,-200.0,573.0,-384.0,174.0,174.0,-384.0,-85.7,86.5,-98.6,-98.6,86.5,238.0,-101.0,18.9,18.9,-101.0,698.0,-536.0,271.0,271.0,-536.0,175.0,-172.0,41.6,41.6,-172.0,703.0,-405.0,112.0,112.0,-405.0,894.0,-611.0,258.0,258.0,-611.0,305.0,-226.0,17.9,17.9,-226.0,374.0,-210.0,78.8,78.8,-210.0,984.0,-695.0,253.0,253.0,-695.0,271.0,-149.0,-17.9,-17.9,-149.0,436.0,-237.0,64.1,64.1,-237.0,954.0,-702.0,311.0,311.0,-702.0,427.0,-328.0,60.9,60.9,-328.0,646.0,-412.0,145.0,145.0,-412.0,840.0,-582.0,223.0,223.0,-582.0,402.0,-318.0,105.0,105.0,-318.0,538.0,-371.0,170.0,170.0,-371.0,1030.0,-763.0,320.0,320.0,-763.0,361.0,-203.0,-9.47,-9.47,-203.0,691.0,-396.0,125.0,125.0,-396.0,1040.0,-720.0,302.0,302.0,-720.0,125.0,-76.2,-16.7,-16.7,-76.2,702.0,-353.0,131.0,131.0,-353.0,1280.0,-775.0,273.0,273.0,-775.0,520.0,-337.0,120.0,120.0,-337.0,810.0,-503.0,237.0,237.0,-503.0,1160.0,-714.0,307.0,307.0,-714.0,399.0,-212.0,69.5,69.5,-212.0,513.0,-296.0,181.0,181.0,-296.0,1000.0,-692.0,337.0,337.0,-692.0,402.0,-225.0,46.3,46.3,-225.0,743.0,-406.0,157.0,157.0,-406.0,1150.0,-721.0,304.0,304.0,-721.0,424.0,-246.0,98.2,98.2,-246.0,540.0,-314.0,172.0,172.0,-314.0,805.0,-497.0,246.0,246.0,-497.0,377.0,-125.0,-2.65,-2.65,-125.0,611.0,-306.0,130.0,130.0,-306.0,1110.0,-715.0,318.0,318.0,-715.0,336.0,-220.0,86.6,86.6,-220.0,741.0,-428.0,194.0,194.0,-428.0,892.0,-546.0,235.0,235.0,-546.0,192.0,-114.0,20.7,20.7,-114.0,626.0,-351.0,113.0,113.0,-351.0,927.0,-587.0,211.0,211.0,-587.0,197.0,-63.8,-55.6,-55.6,-63.8,346.0,-197.0,93.7,93.7,-197.0,932.0,-597.0,244.0,244.0,-597.0,100.0,-65.5,-15.0,-15.0,-65.5,393.0,-159.0,91.0,91.0,-159.0,912.0,-552.0,271.0,271.0,-552.0,81.0,-65.1,-54.3,-54.3,-65.1,299.0,-147.0,-1.85,-1.85,-147.0,659.0,-558.0,170.0,170.0,-558.0,136.0,-181.0,-72.7,-72.7,-181.0,310.0,-306.0,40.4,40.4,-306.0,484.0,-439.0,87.5,87.5,-439.0,-65.8,-41.2,-121.0,-121.0,-41.2,182.0,-148.0,-3.55,-3.55,-148.0,859.0,-676.0,202.0,202.0,-676.0,13.0,30.4,297.0,20.6,23.1,-5.86,-4.5,-187.0,0.797,2.99,10.3,29.5,300.0,19.9,19.1,21.4,35.7,479.0,20.5,24.1,4.54,25.5,250.0,18.6,24.2,15.4,30.7,157.0,23.3,25.0,-11.2,-6.22,-179.0,-0.624,-4.96,5.72,3.98,49.8,1.24,-5.13,-5.17,-1.21,142.0,-3.41,-5.88,16.9,10.2,229.0,1.86,-0.165,5.99,5.01,322.0,-2.78,-0.917,-10.9,-5.2,92.4,-4.65,-0.752,28.6,7.41,327.0,13.6,31.9,-9.73,-5.06,-53.3,-9.39,-1.26,-12200.0,146.0,2740000.0,-3470.0,1570.0,2010000.0,11500.0,27100000000.0,238000.0,2900000.0,-82200000.0,44700.0,2230000000000.0,-10000000.0,-861000.0,9340000000.0,3300000.0,9110000000000000.0,560000000.0,11900000000.0,69.5,51.6,1050.0,52.1,99.8,-24.5,-15.8,-320.0,-7.75,-6.46,39.9,39.9,951.0,37.4,85.3,69.5,51.6,1050.0,52.1,99.8,31.2,32.9,729.0,33.9,91.7,45.0,35.9,641.0,44.4,93.4,-29.6,-11.7,-98.4,-14.7,-14.5,8.64,6.98,221.0,3.47,-6.38,-5.14,4.05,310.0,-6.98,-8.01,38.2,18.7,320.0,18.2,8.08,24.5,15.8,408.0,7.75,6.46,-13.8,-2.93,88.5,-10.4,-1.62,-77.2,10.3,-387.0,-37.3,-56.7,29.4,7.59,81.0,36.0,2.74,-32.0,25.7,-387.0,4.07,-56.7,-77.2,10.3,-32.8,-37.3,-27.6,-47.8,17.9,-256.0,-1.29,-49.9,-21.9,26.5,-306.0,8.32,-54.0,45.3,15.5,-355.0,41.4,-29.1,15.8,7.88,-132.0,5.36,-6.75,-10.0,-0.741,-81.0,-4.25,-2.74,-29.4,-7.59,223.0,-36.0,22.3,-55.3,-16.2,274.0,-45.6,26.3,-25.9,-8.62,50.6,-9.61,4.01,22100.0,-218.0,-1030.0,-10000.0,-10800.0,-4010.0,-2420.0,11200.0,3120.0,5220.0,49.9,-3410.0,-218.0,10300.0,-666.0,-18500.0,-3140.0,-801.0,-1420.0,681.0,-3050.0,392.0,-30.9,-1150.0,-1030.0,-666.0,1820.0,-2390.0,10700.0,-345.0,-825.0,-658.0,666.0,-4940.0,313.0,-403.0,-10000.0,-18500.0,-2390.0,76400.0,-16100.0,872.0,-5990.0,-8820.0,-12500.0,5440.0,-4650.0,-1570.0,-10800.0,-3140.0,10700.0,-16100.0,109000.0,-2950.0,-8210.0,-7030.0,-281.0,-41500.0,-428.0,-4770.0,-4010.0,-801.0,-345.0,872.0,-2950.0,38000.0,-5390.0,-325.0,4320.0,908.0,-2200.0,-1140.0,-2420.0,-1420.0,-825.0,-5990.0,-8210.0,-5390.0,14600.0,-2500.0,4820.0,4210.0,-675.0,8120.0,11200.0,681.0,-658.0,-8820.0,-7030.0,-325.0,-2500.0,14100.0,1090.0,3850.0,704.0,-2780.0,3120.0,-3050.0,666.0,-12500.0,-281.0,4320.0,4820.0,1090.0,14900.0,1540.0,-170.0,3600.0,5220.0,392.0,-4940.0,5440.0,-41500.0,908.0,4210.0,3850.0,1540.0,17700.0,212.0,2340.0,49.9,-30.9,313.0,-4650.0,-428.0,-2200.0,-675.0,704.0,-170.0,212.0,6790.0,-735.0,-3410.0,-1150.0,-403.0,-1570.0,-4770.0,-1140.0,8120.0,-2780.0,3600.0,2340.0,-735.0,7590.0,134000.0,84500.0,40200.0,30400.0,21200.0,7840.0,6800.0,5050.0,3.41e-13,200.0,2090.0,1940.0,9.15,-4.08,-7.61,8.73,10.5,10.4,7.57,9.0,8.03,4.6,7.66,8.19,-2.65,-12.7,-5.25,-1.36,-0.285,-0.643,-0.652,-0.478,-1.35,-1.46,0.0677,-2.43,-6.14,-2.67,-0.424,-1.07,-0.0931,-1.06,-0.249,-1.2,0.403,-1.3,-2.92,-1.0,-2.06,-0.364,-0.0469,-1.59,-0.381,0.471,-0.728,-1.02,-4.23,-0.852,-0.768,-0.409,-1.38,-0.134,-0.406,-4.54,-1.18,-2.08,-2.73,-0.429,1.12,-1.04,-1.48,-3.46,-2.16,-0.818,0.00922,0.388,-4.4,-7.35,-1.78,0.0264,-0.562,-5.54,-3.41,0.195,-0.891,-3.87,0.687,-0.822,0.509,-0.174,0.0,4.98,0.0,0.0,0.0,-35800.0,-54700.0,230000.0,55600.0,10500.0,232000.0,-31500.0,-40100.0,148000.0,2890.0,4160.0,353000.0,4240.0,35800.0,328000.0,-8340.0,54700.0,233000.0,-897.0,194000.0,501000.0,-24200.0,59700.0,525000.0,-8200.0,114000.0,457000.0,-6250.0,144000.0,435000.0,-93300.0,-82400.0,597000.0,9890.0,187000.0,374000.0,-844.0,95800.0,354000.0,-112000.0,78900.0,513000.0,5550.0,68400.0,362000.0,-87000.0,59600.0,507000.0,-55900.0,64400.0,331000.0,-20300.0,16400.0,366000.0,-23300.0,110000.0,377000.0,-34400.0,39200.0,243000.0,-24100.0,80900.0,430000.0,-10900.0,28100.0,487000.0,-8790.0,56000.0,319000.0,-22800.0,127000.0,259000.0,1140.0,11300.0,379000.0,-347.0,300.0,-178.0,-178.0,300.0,-53.9,170.0,-105.0,-105.0,170.0,493.0,-265.0,88.8,88.8,-265.0,-221.0,182.0,-152.0,-152.0,182.0,168.0,25.6,-52.9,-52.9,25.6,399.0,-238.0,104.0,104.0,-238.0,-321.0,222.0,-87.1,-87.1,222.0,33.8,120.0,-81.9,-81.9,120.0,289.0,-166.0,104.0,104.0,-166.0,-207.0,203.0,-152.0,-152.0,203.0,204.0,-16.2,-51.6,-51.6,-16.2,612.0,-439.0,200.0,200.0,-439.0,5.43,-20.1,-40.6,-40.6,-20.1,330.0,-203.0,74.9,74.9,-203.0,567.0,-381.0,174.0,174.0,-381.0,-79.8,83.4,-99.3,-99.3,83.4,232.0,-97.4,19.5,19.5,-97.4,704.0,-539.0,271.0,271.0,-539.0,169.0,-169.0,42.0,42.0,-169.0,708.0,-409.0,112.0,112.0,-409.0,889.0,-607.0,258.0,258.0,-607.0,311.0,-229.0,17.9,17.9,-229.0,369.0,-206.0,78.7,78.7,-206.0,989.0,-699.0,254.0,254.0,-699.0,265.0,-145.0,-18.1,-18.1,-145.0,442.0,-241.0,64.3,64.3,-241.0,948.0,-698.0,310.0,310.0,-698.0,433.0,-332.0,61.4,61.4,-332.0,641.0,-408.0,144.0,144.0,-408.0,846.0,-586.0,224.0,224.0,-586.0,396.0,-313.0,104.0,104.0,-313.0,543.0,-375.0,171.0,171.0,-375.0,1020.0,-758.0,319.0,319.0,-758.0,366.0,-208.0,-8.59,-8.59,-208.0,686.0,-391.0,124.0,124.0,-391.0,1040.0,-725.0,303.0,303.0,-725.0,119.0,-71.2,-17.8,-17.8,-71.2,708.0,-358.0,133.0,133.0,-358.0,1280.0,-770.0,272.0,272.0,-770.0,526.0,-342.0,122.0,122.0,-342.0,804.0,-497.0,236.0,236.0,-497.0,1170.0,-720.0,309.0,309.0,-720.0,393.0,-207.0,68.0,68.0,-207.0,519.0,-302.0,182.0,182.0,-302.0,995.0,-686.0,335.0,335.0,-686.0,408.0,-231.0,48.1,48.1,-231.0,736.0,-399.0,155.0,155.0,-399.0,1160.0,-727.0,306.0,306.0,-727.0,417.0,-239.0,96.1,96.1,-239.0,547.0,-321.0,174.0,174.0,-321.0,798.0,-490.0,244.0,244.0,-490.0,384.0,-132.0,-0.181,-0.181,-132.0,603.0,-298.0,127.0,127.0,-298.0,1120.0,-723.0,321.0,321.0,-723.0,329.0,-212.0,83.8,83.8,-212.0,749.0,-436.0,197.0,197.0,-436.0,884.0,-538.0,231.0,231.0,-538.0,200.0,-122.0,24.0,24.0,-122.0,618.0,-342.0,109.0,109.0,-342.0,936.0,-597.0,215.0,215.0,-597.0,188.0,-54.1,-59.6,-59.6,-54.1,356.0,-207.0,98.0,98.0,-207.0,922.0,-586.0,239.0,239.0,-586.0,110.0,-76.9,-10.1,-10.1,-76.9,382.0,-147.0,85.7,85.7,-147.0,924.0,-565.0,277.0,277.0,-565.0,68.7,-51.4,-60.6,-60.6,-51.4,312.0,-161.0,5.02,5.02,-161.0,645.0,-542.0,162.0,162.0,-542.0,152.0,-199.0,-64.1,-64.1,-199.0,292.0,-286.0,30.5,30.5,-286.0,505.0,-463.0,99.3,99.3,-463.0,-91.5,-11.4,-136.0,-136.0,-11.4,217.0,-188.0,15.8,15.8,-188.0,803.0,-612.0,171.0,171.0,-612.0,79.7,-120.0,-39.4,-39.4,-120.0,373.0,-270.0,47.8,47.8,-270.0,670.0,-493.0,165.0,165.0,-493.0,-200.0,128.0,-145.0,-145.0,128.0,335.0,-176.0,34.9,34.9,-176.0,837.0,-563.0,206.0,206.0,-563.0,121.0,-111.0,-38.0,-38.0,-111.0,404.0,-278.0,103.0,103.0,-278.0,853.0,-520.0,145.0,145.0,-520.0,7.98,-27.8,-27.6,-27.6,-27.8,121.0,-67.6,55.0,55.0,-67.6,718.0,-505.0,193.0,193.0,-505.0,-194.0,79.9,-56.1,-56.1,79.9,151.0,-102.0,92.4,92.4,-102.0,718.0,-511.0,211.0,211.0,-511.0,166.0,-61.8,-39.9,-39.9,-61.8,469.0,-261.0,104.0,104.0,-261.0,399.0,-270.0,124.0,124.0,-270.0,70.5,0.756,-85.8,-85.8,0.756,464.0,-234.0,59.0,59.0,-234.0,729.0,-489.0,183.0,183.0,-489.0,-61.3,68.5,-91.5,-91.5,68.5,333.0,-142.0,19.1,19.1,-142.0,673.0,-452.0,193.0,193.0,-452.0,3.86,50.1,-118.0,-118.0,50.1,419.0,-249.0,117.0,117.0,-249.0,643.0,-399.0,134.0,134.0,-399.0,20.0,-5.92,-46.4,-46.4,-5.92,384.0,-174.0,37.0,37.0,-174.0,667.0,-464.0,182.0,182.0,-464.0,-215.0,186.0,-118.0,-118.0,186.0,-75.7,156.0,-55.7,-55.7,156.0,766.0,-462.0,157.0,157.0,-462.0,103.0,-23.0,-72.1,-72.1,-23.0,494.0,-229.0,37.7,37.7,-229.0,480.0,-308.0,129.0,129.0,-308.0,18.2,-2.14,-46.5,-46.5,-2.14,258.0,-113.0,53.7,53.7,-113.0,419.0,-258.0,106.0,106.0,-258.0,-158.0,194.0,-154.0,-154.0,194.0,219.0,-69.2,21.9,21.9,-69.2,648.0,-399.0,159.0,159.0,-399.0,134.0,3.53,-104.0,-104.0,3.53,282.0,-103.0,23.2,23.2,-103.0,525.0,-299.0,96.5,96.5,-299.0,-256.0,216.0,-128.0,-128.0,216.0,190.0,-50.3,29.6,29.6,-50.3,626.0,-403.0,146.0,146.0,-403.0,-124.0,125.0,-116.0,-116.0,125.0,234.0,-81.1,35.4,35.4,-81.1,631.0,-380.0,120.0,120.0,-380.0,-39.9,87.3,-117.0,-117.0,87.3,142.0,0.38,-16.7,-16.7,0.38,460.0,-295.0,132.0,132.0,-295.0,-65.4,58.1,-80.2,-80.2,58.1,252.0,-130.0,45.1,45.1,-130.0,620.0,-391.0,136.0,136.0,-391.0,-263.0,214.0,-120.0,-120.0,214.0,163.0,-42.0,39.5,39.5,-42.0,367.0,-228.0,110.0,110.0,-228.0,-307.0,198.0,-80.5,-80.5,198.0,364.0,-201.0,90.8,90.8,-201.0,673.0,-420.0,163.0,163.0,-420.0,-52.8,113.0,-118.0,-118.0,113.0,211.0,-43.9,22.0,22.0,-43.9,819.0,-469.0,144.0,144.0,-469.0,-181.0,215.0,-145.0,-145.0,215.0,398.0,-117.0,18.0,18.0,-117.0,583.0,-374.0,195.0,195.0,-374.0,-54.5,94.4,-103.0,-103.0,94.4,531.0,-258.0,84.0,84.0,-258.0,582.0,-359.0,148.0,148.0,-359.0,-4.04,33.8,-27.1,-27.1,33.8,122.0,-16.0,34.1,34.1,-16.0,622.0,-368.0,155.0,155.0,-368.0"
            happy_sample = "5.21,28.4,18.5,3.66,22.6,-0.119,-2.35,5.07,0.461,-4.26,6.18,31.5,15.2,3.11,19.8,4.39,27.8,16.8,3.74,29.5,2.07,27.4,25.5,3.59,14.7,8.16,27.1,16.8,4.18,26.0,1.79,3.76,-1.54,-0.625,-9.73,4.11,4.11,-10.3,-0.48,5.07,-1.98,4.4,-1.54,-1.07,-6.26,2.32,0.354,-8.74,0.144,14.8,-3.77,0.643,0.003,-0.443,3.47,-6.08,0.289,8.75,-0.587,-11.3,20.1,4.07,11.5,6.38,33.2,0.917,-0.615,0.96,0.69,3.99,401.0,10.9,616.0,30.4,-12300.0,295000.0,690.0,47500.0,4450.0,3070000.0,1040000.0,1040.0,539000.0,14600.0,-101000000.0,266000000.0,41000.0,26400000.0,751000.0,13600000000.0,43.5,38.7,51.6,21.8,81.9,2.4,-3.8,10.1,2.34,-0.637,39.8,36.9,38.9,16.9,77.5,41.1,38.7,41.5,19.5,81.9,33.6,34.9,51.6,16.6,81.2,43.5,34.9,43.8,21.8,76.6,-1.33,-1.8,-2.65,-2.62,-4.33,6.2,1.99,-12.7,0.31,-3.7,-3.73,2.04,-4.86,-4.95,0.967,7.52,3.8,-10.1,2.93,0.637,-2.4,3.84,-2.21,-2.34,5.3,-9.92,0.0469,7.87,-5.26,4.66,-30.7,19.0,-6.05,-10.6,-75.3,-0.167,1.4,6.61,1.36,-28.5,-27.2,26.3,-3.13,-10.6,-46.7,-30.6,19.0,-6.05,-7.23,-33.2,-30.7,21.3,1.72,-7.44,-75.3,-20.7,20.4,0.551,-9.24,-52.8,3.31,7.31,2.93,-3.38,-13.6,3.48,5.05,-4.85,-3.17,28.5,-6.58,5.91,-3.68,-1.36,6.05,0.167,-2.26,-7.77,0.206,42.1,-9.9,-1.4,-6.61,2.01,19.6,-10.1,0.862,1.17,1.81,-22.5,131.0,-22.0,29.3,-58.2,-180.0,-60.0,-22.4,96.7,75.6,101.0,-68.2,52.9,-22.0,116.0,4.07,-120.0,-6.64,13.1,0.349,11.9,-5.85,-103.0,1.28,-45.0,29.3,4.07,29.2,-33.2,-30.5,-69.4,-6.06,24.5,-17.5,41.0,-30.8,2.3,-58.2,-120.0,-33.2,320.0,42.9,-101.0,7.32,-67.0,53.1,112.0,31.1,-71.5,-180.0,-6.64,-30.5,42.9,500.0,16.7,29.9,-168.0,-40.6,-405.0,43.8,6.47,-60.0,13.1,-69.4,-101.0,16.7,690.0,-3.88,-43.0,-154.0,-113.0,177.0,143.0,-22.4,0.349,-6.06,7.32,29.9,-3.88,23.2,-5.43,-54.7,22.4,9.12,-14.6,96.7,11.9,24.5,-67.0,-168.0,-43.0,-5.43,155.0,4.03,91.1,-33.4,-53.2,75.6,-5.85,-17.5,53.1,-40.6,-154.0,-54.7,4.03,433.0,-216.0,-66.1,43.0,101.0,-103.0,41.0,112.0,-405.0,-113.0,22.4,91.1,-216.0,736.0,-36.2,-95.3,-68.2,1.28,-30.8,31.1,43.8,177.0,9.12,-33.4,-66.1,-36.2,95.3,-5.59,52.9,-45.0,2.3,-71.5,6.47,143.0,-14.6,-53.2,43.0,-95.3,-5.59,263.0,1200.0,841.0,592.0,397.0,254.0,106.0,52.6,8.25e-14,3.54,5.81,24.7,19.8,2.47,-3.98,-1.53,4.7,0.92,6.15,-4.8,0.762,5.42,3.26,2.9,3.18,-2.04,5.31,0.947,-1.31,0.189,-0.675,5.13,-0.747,-1.34,-1.24,-1.02,1.23,-2.84,3.92,-0.154,5.79,0.246,0.414,-3.08,-0.457,-2.52,-0.802,-6.33,-0.875,1.8,-4.46,-0.356,4.45,-1.14,-0.363,-1.86,0.407,-5.88,-1.6,-1.16,0.0387,1.82,-3.2,-0.00652,7.5,3.68,0.123,-4.52,0.909,3.79,1.05,-5.28,0.2,-0.739,-1.53,0.644,0.144,-1.01,3.15,-0.115,-2.99,0.292,-5.04,1.07,-1.28,-0.604,-1.93,2.83,-1.29,-4.41,-0.0764,0.0,5.0,0.0,0.0,0.0,1910.0,884.0,1990.0,950.0,-198.0,159.0,2720.0,1350.0,3140.0,2070.0,3690.0,1510.0,1040.0,905.0,2760.0,2640.0,1000.0,-475.0,1100.0,2810.0,1400.0,303.0,2280.0,2440.0,1680.0,5810.0,4010.0,2760.0,1490.0,2290.0,2140.0,1390.0,5070.0,2370.0,910.0,3540.0,78.0,1230.0,3520.0,1890.0,660.0,1140.0,2000.0,708.0,597.0,756.0,1150.0,1830.0,3710.0,1170.0,2920.0,2640.0,1090.0,3050.0,3150.0,813.0,4760.0,1280.0,1590.0,2940.0,2920.0,3330.0,4650.0,2830.0,3920.0,4890.0,5650.0,480.0,2280.0,1250.0,1180.0,3700.0,4630.0,-2670.0,1250.0,85.5,2.37,-40.5,-40.5,2.37,-15.3,-18.0,-42.5,-42.5,-18.0,93.6,17.7,-0.574,-0.574,17.7,83.9,6.24,-31.6,-31.6,6.24,71.8,2.73,-78.8,-78.8,2.73,120.0,19.8,-26.8,-26.8,19.8,95.1,10.3,-25.3,-25.3,10.3,70.1,-14.2,-83.9,-83.9,-14.2,111.0,19.3,-30.8,-30.8,19.3,70.7,4.85,-30.2,-30.2,4.85,71.4,-2.29,-81.7,-81.7,-2.29,97.9,27.5,-18.6,-18.6,27.5,71.9,22.4,-17.3,-17.3,22.4,75.6,-0.651,-52.1,-52.1,-0.651,77.2,22.8,38.0,38.0,22.8,95.3,8.14,-8.6,-8.6,8.14,67.6,-13.4,-58.7,-58.7,-13.4,133.0,21.6,-25.8,-25.8,21.6,75.4,12.2,-13.8,-13.8,12.2,-10.1,-23.4,-3.49,-3.49,-23.4,158.0,49.5,-46.6,-46.6,49.5,33.8,10.7,1.13,1.13,10.7,43.2,-7.12,-61.7,-61.7,-7.12,77.5,21.5,6.55,6.55,21.5,1.94,4.37,24.4,24.4,4.37,54.4,9.42,-100.0,-100.0,9.42,118.0,24.5,-7.02,-7.02,24.5,131.0,26.6,-55.5,-55.5,26.6,-3.05,-24.0,-32.2,-32.2,-24.0,80.8,5.86,17.7,17.7,5.86,143.0,19.3,-68.1,-68.1,19.3,11.9,-13.2,-36.3,-36.3,-13.2,157.0,22.7,-36.7,-36.7,22.7,102.0,3.67,-32.4,-32.4,3.67,3.3,-30.0,-29.5,-29.5,-30.0,118.0,27.8,-23.8,-23.8,27.8,31.2,-11.2,16.3,16.3,-11.2,18.4,-8.72,-58.3,-58.3,-8.72,105.0,22.4,-1.49,-1.49,22.4,80.4,-0.142,-23.8,-23.8,-0.142,20.4,-22.3,-27.8,-27.8,-22.3,64.4,9.35,26.8,26.8,9.35,115.0,18.7,-56.9,-56.9,18.7,50.5,-12.3,-59.1,-59.1,-12.3,49.9,13.8,25.7,25.7,13.8,50.0,18.0,-23.4,-23.4,18.0,43.4,-1.52,-64.6,-64.6,-1.52,85.9,27.6,5.94,5.94,27.6,119.0,24.2,-65.7,-65.7,24.2,18.0,-18.5,-46.1,-46.1,-18.5,38.3,-1.18,30.1,30.1,-1.18,128.0,18.1,-84.2,-84.2,18.1,11.4,-19.9,-39.3,-39.3,-19.9,128.0,26.8,-28.8,-28.8,26.8,114.0,21.2,-71.2,-71.2,21.2,3.26,-6.75,-36.2,-36.2,-6.75,134.0,17.8,-16.5,-16.5,17.8,79.7,-10.3,-17.0,-17.0,-10.3,72.2,-10.8,-57.6,-57.6,-10.8,110.0,-2.07,3.17,3.17,-2.07,137.0,7.98,-59.3,-59.3,7.98,73.3,-24.1,-67.5,-67.5,-24.1,138.0,27.4,-23.9,-23.9,27.4,98.1,10.1,-36.1,-36.1,10.1,105.0,-4.9,-85.0,-85.0,-4.9,160.0,24.5,-19.6,-19.6,24.5,117.0,19.9,-56.9,-56.9,19.9,46.6,-18.8,-53.4,-53.4,-18.8,158.0,37.6,-48.6,-48.6,37.6,61.7,2.09,-10.6,-10.6,2.09,33.6,-12.2,-33.4,-33.4,-12.2,114.0,29.1,16.6,16.6,29.1,80.2,7.98,-65.1,-65.1,7.98,56.9,-9.7,-95.1,-95.1,-9.7,101.0,8.8,-28.0,-28.0,8.8,55.4,-0.62,-56.9,-56.9,-0.62,52.0,-19.5,-83.3,-83.3,-19.5,74.5,18.8,5.24,5.24,18.8,29.5,-5.76,-12.0,-12.0,-5.76,-33.0,-28.4,-16.0,-16.0,-28.4,25.8,14.1,53.9,53.9,14.1,114.0,21.2,-70.0,-70.0,21.2,4.53,-8.99,-39.9,-39.9,-8.99,120.0,16.3,-2.69,-2.69,16.3,110.0,3.25,-58.5,-58.5,3.25,73.3,-27.4,-73.8,-73.8,-27.4,70.3,0.778,22.1,22.1,0.778,53.6,-7.74,-14.4,-14.4,-7.74,-1.28,-26.4,-31.9,-31.9,-26.4,85.1,-11.4,23.6,23.6,-11.4,141.0,5.04,-77.4,-77.4,5.04,18.3,-24.8,-23.1,-23.1,-24.8,31.9,-14.3,82.3,82.3,-14.3,44.9,-27.5,11.6,11.6,-27.5,-3.2,-36.8,-9.37,-9.37,-36.8,83.5,1.6,34.2,34.2,1.6,128.0,-1.44,-55.1,-55.1,-1.44,56.7,-15.9,-64.5,-64.5,-15.9,168.0,23.7,-31.1,-31.1,23.7,67.1,-20.0,3.83,3.83,-20.0,140.0,-7.77,-92.0,-92.0,-7.77,141.0,7.84,5.68,5.68,7.84,84.9,-8.11,-29.8,-29.8,-8.11,30.2,-37.1,-16.2,-16.2,-37.1,79.9,-6.83,22.4,22.4,-6.83,69.8,-38.3,-23.9,-23.9,-38.3,106.0,-37.4,-73.9,-73.9,-37.4,139.0,-4.49,1.01,1.01,-4.49,82.9,-13.7,-47.6,-47.6,-13.7,44.3,-11.2,-60.9,-60.9,-11.2,125.0,17.8,-9.34,-9.34,17.8,97.1,-1.76,-43.0,-43.0,-1.76,28.6,-15.1,-42.8,-42.8,-15.1,159.0,35.0,-47.0,-47.0,35.0,88.0,4.9,-57.4,-57.4,4.9,83.1,2.09,-77.1,-77.1,2.09,182.0,27.1,-35.3,-35.3,27.1,74.0,-10.3,-22.9,-22.9,-10.3,-26.0,-39.1,3.23,3.23,-39.1,138.0,34.1,-22.3,-22.3,34.1,20.6,-17.3,-9.26,-9.26,-17.3,7.24,-18.6,-36.6,-36.6,-18.6,65.2,12.0,6.07,6.07,12.0,72.4,4.73,-60.2,-60.2,4.73,46.1,-1.45,-65.5,-65.5,-1.45,41.5,-2.51,44.1,44.1,-2.51,58.1,-1.13,-39.1,-39.1,-1.13,42.4,5.0,-74.3,-74.3,5.0,97.9,21.6,-13.2,-13.2,21.6,64.9,7.15,-49.0,-49.0,7.15,60.4,0.729,-72.4,-72.4,0.729,135.0,45.9,-35.5,-35.5,45.9,63.3,9.14,-29.3,-29.3,9.14,25.3,0.788,-48.4,-48.4,0.788,110.0,25.0,-1.12,-1.12,25.0,84.5,0.697,-26.8,-26.8,0.697,85.0,5.15,-70.1,-70.1,5.15,149.0,25.2,6.75,6.75,25.2,107.0,8.56,-42.3,-42.3,8.56,91.3,5.39,-50.3,-50.3,5.39,141.0,39.2,-1.15,-1.15,39.2,137.0,25.7,-65.7,-65.7,25.7,16.6,-12.3,0.424,0.424,-12.3,102.0,24.2,33.8,33.8,24.2,92.2,21.2,-39.1,-39.1,21.2,113.0,7.54,-61.0,-61.0,7.54,150.0,34.6,-1.83,-1.83,34.6,142.0,19.2,-57.4,-57.4,19.2,-11.3,-26.7,28.3,28.3,-26.7,90.1,-16.9,53.8,53.8,-16.9,5.97,28.6,21.2,4.46,20.3,1.57,2.56,0.345,1.07,-0.0563,2.14,27.5,25.5,3.69,14.3,8.14,27.1,16.7,4.16,26.2,8.65,28.0,19.3,6.44,18.7,4.91,31.6,23.4,3.59,21.7,-6.0,0.322,8.77,-0.475,-11.9,-6.51,-0.584,6.19,-2.76,-4.45,-2.77,-4.17,2.06,0.0972,-7.46,-0.512,-0.906,-2.58,-2.28,7.46,3.24,-4.49,-6.71,0.572,4.44,3.75,-3.59,-4.13,2.85,-3.01,20.2,3.81,10.6,6.14,32.6,-0.633,0.109,-2.18,-1.16,-6.1,2930.0,-14.2,512.0,6.2,-4600.0,296000.0,398.0,33600.0,3720.0,3120000.0,3800000.0,-831.0,389000.0,4210.0,-55000000.0,268000000.0,14000.0,15800000.0,546000.0,14800000000.0,43.6,34.9,51.4,21.5,94.9,-2.51,-0.167,-3.85,-4.3,14.9,33.9,34.9,51.4,16.6,79.9,43.6,34.9,42.4,21.5,78.5,40.3,34.3,39.2,17.2,75.9,41.1,34.8,47.6,16.5,94.9,-9.73,-0.0295,9.03,-4.9,1.46,-6.45,0.625,12.3,-0.595,3.99,-7.21,0.138,3.85,0.141,-14.9,3.27,0.654,3.23,4.3,2.53,2.51,0.167,-5.18,5.04,-16.4,-0.76,-0.487,-8.4,0.737,-18.9,-31.2,20.7,0.188,-9.23,-76.0,12.7,0.484,-0.63,3.0,28.4,-31.2,20.7,2.05,-7.58,-76.0,-20.9,21.3,0.818,-9.23,-61.9,-16.5,21.1,6.73,-5.07,-29.3,-18.5,26.1,0.188,-6.23,-47.6,-10.3,-0.632,1.23,1.64,-14.1,-14.7,-0.484,-4.69,-2.51,-46.7,-12.7,-5.45,1.86,-1.35,-28.4,-4.44,0.148,-5.92,-4.15,-32.6,-2.38,-4.81,0.63,-3.0,-14.3,2.06,-4.96,6.55,1.16,18.3,109.0,-23.3,3.63,-36.3,-121.0,-70.5,-14.9,14.1,-42.9,58.2,73.4,-6.94,-23.3,134.0,-23.7,-114.0,41.7,-73.0,5.75,-41.9,-41.0,-40.5,47.8,2.87,3.63,-23.7,22.4,12.8,-60.3,36.6,-2.81,15.2,-28.6,47.7,-50.0,13.2,-36.3,-114.0,12.8,364.0,3.38,-151.0,47.9,-121.0,196.0,57.7,-32.6,91.2,-121.0,41.7,-60.3,3.38,446.0,88.3,2.29,8.48,89.0,-273.0,59.5,-26.8,-70.5,-73.0,36.6,-151.0,88.3,904.0,-9.15,172.0,-124.0,14.0,-258.0,-141.0,-14.9,5.75,-2.81,47.9,2.29,-9.15,32.1,-17.4,54.3,-20.2,-12.9,16.2,14.1,-41.9,15.2,-121.0,8.48,172.0,-17.4,175.0,-81.1,-40.5,-49.7,0.426,-42.9,-41.0,-28.6,196.0,89.0,-124.0,54.3,-81.1,880.0,-301.0,5.61,19.7,58.2,-40.5,47.7,57.7,-273.0,14.0,-20.2,-40.5,-301.0,334.0,-58.6,28.7,73.4,47.8,-50.0,-32.6,59.5,-258.0,-12.9,-49.7,5.61,-58.6,204.0,2.11,-6.94,2.87,13.2,91.2,-26.8,-141.0,16.2,0.426,19.7,28.7,2.11,146.0,1240.0,1010.0,626.0,375.0,185.0,166.0,86.7,39.4,17.3,8.09,-8.29e-16,0.319,1.03,-0.335,-12.9,1.08,3.97,6.18,0.172,2.46,6.43,3.6,-1.34,3.55,2.51,-7.16,-6.41,2.63,0.979,0.944,2.03,-0.00948,-1.13,1.72,2.28,6.17,-5.21,3.37,-1.41,-2.15,-0.382,-0.328,0.699,0.274,-0.651,2.61,2.49,-1.86,3.77,1.51,0.0303,-2.01,-3.41,-0.134,-2.51,-1.35,4.16,-3.46,-0.406,0.691,2.93,1.89,0.805,3.7,-4.83,0.85,-2.31,-2.01,-1.33,-2.1,-3.63,0.348,2.39,3.04,0.134,2.28,0.283,2.78,-4.52,-1.12,-0.267,1.77,-9.7,2.48,-1.13,-4.35,3.98,4.89,2.41,-1.64,0.0,5.0,4.88,0.0,0.0,470.0,2030.0,1860.0,663.0,470.0,2150.0,282.0,856.0,4000.0,-395.0,1270.0,2350.0,972.0,902.0,2610.0,3660.0,1020.0,559.0,576.0,440.0,3010.0,4870.0,1690.0,4590.0,1590.0,3840.0,3110.0,928.0,1210.0,2880.0,1660.0,4940.0,4000.0,2720.0,1700.0,5290.0,943.0,1270.0,7130.0,3750.0,6850.0,9460.0,1350.0,-540.0,4510.0,838.0,1440.0,2130.0,2630.0,2480.0,1670.0,1130.0,2780.0,2280.0,750.0,1490.0,5340.0,1070.0,1170.0,1690.0,1530.0,1400.0,4960.0,1060.0,341.0,5870.0,2880.0,821.0,3070.0,2740.0,1260.0,4690.0,3090.0,-518.0,3830.0,43.2,-7.94,-35.8,-35.8,-7.94,58.6,-15.8,-91.5,-91.5,-15.8,70.0,16.3,10.2,10.2,16.3,32.8,-3.89,-15.3,-15.3,-3.89,-35.5,-29.9,-13.6,-13.6,-29.9,27.7,15.3,52.3,52.3,15.3,113.0,20.3,-68.8,-68.8,20.3,5.66,-8.25,-40.7,-40.7,-8.25,120.0,15.7,-2.27,-2.27,15.7,111.0,3.74,-58.7,-58.7,3.74,72.9,-27.8,-73.8,-73.8,-27.8,70.5,1.09,22.3,22.3,1.09,53.5,-7.98,-14.8,-14.8,-7.98,-1.3,-26.2,-31.4,-31.4,-26.2,85.3,-11.5,23.0,23.0,-11.5,140.0,5.13,-76.6,-76.6,5.13,18.6,-24.9,-23.9,-23.9,-24.9,31.5,-14.3,83.2,83.2,-14.3,45.4,-27.4,10.6,10.6,-27.4,-3.76,-36.8,-8.33,-8.33,-36.8,84.2,1.68,33.1,33.1,1.68,127.0,-1.55,-53.9,-53.9,-1.55,57.5,-15.7,-65.7,-65.7,-15.7,167.0,23.5,-29.9,-29.9,23.5,67.9,-19.8,2.51,2.51,-19.8,139.0,-7.97,-90.6,-90.6,-7.97,141.0,8.06,4.27,4.27,8.06,83.9,-8.34,-28.3,-28.3,-8.34,31.2,-36.8,-17.7,-17.7,-36.8,78.8,-7.1,23.9,23.9,-7.1,70.9,-38.0,-25.4,-25.4,-38.0,105.0,-37.7,-72.3,-72.3,-37.7,141.0,-4.17,-0.624,-0.624,-4.17,81.6,-14.0,-46.0,-46.0,-14.0,45.6,-10.8,-62.6,-62.6,-10.8,124.0,17.5,-7.6,-7.6,17.5,98.5,-1.37,-44.7,-44.7,-1.37,27.1,-15.5,-41.0,-41.0,-15.5,161.0,35.4,-48.8,-48.8,35.4,86.5,4.46,-55.5,-55.5,4.46,84.6,2.55,-79.0,-79.0,2.55,180.0,26.6,-33.3,-33.3,26.6,75.6,-9.77,-24.8,-24.8,-9.77,-27.8,-39.6,5.24,5.24,-39.6,139.0,34.6,-24.3,-24.3,34.6,18.8,-17.8,-7.17,-7.17,-17.8,9.1,-18.1,-38.8,-38.8,-18.1,63.3,11.4,8.24,8.24,11.4,74.4,5.33,-62.5,-62.5,5.33,44.0,-2.08,-63.3,-63.3,-2.08,43.6,-1.86,41.8,41.8,-1.86,55.9,-1.8,-36.7,-36.7,-1.8,44.6,5.7,-76.7,-76.7,5.7,95.6,20.9,-10.7,-10.7,20.9,67.3,7.9,-51.6,-51.6,7.9,57.9,-0.0534,-69.8,-69.8,-0.0534,137.0,46.7,-38.2,-38.2,46.7,60.6,8.29,-26.5,-26.5,8.29,28.0,1.68,-51.2,-51.2,1.68,107.0,24.1,1.79,1.79,24.1,87.5,1.68,-29.8,-29.8,1.68,81.9,4.11,-67.0,-67.0,4.11,152.0,26.3,3.5,3.5,26.3,104.0,7.41,-38.9,-38.9,7.41,94.9,6.62,-53.9,-53.9,6.62,137.0,37.9,2.6,2.6,37.9,141.0,27.1,-69.7,-69.7,27.1,12.2,-13.8,4.66,4.66,-13.8,106.0,25.8,29.3,29.3,25.8,87.1,19.3,-34.1,-34.1,19.3,118.0,9.62,-66.5,-66.5,9.62,144.0,32.2,4.33,4.33,32.2,149.0,22.0,-64.6,-64.6,22.0,-19.2,-30.1,37.0,37.0,-30.1,99.2,-12.3,42.0,42.0,-12.3,64.6,-6.07,20.6,20.6,-6.07,60.1,-10.3,-30.5,-30.5,-10.3,80.3,25.3,35.3,35.3,25.3,75.8,-1.64,-34.0,-34.0,-1.64,28.9,-11.9,-34.9,-34.9,-11.9,100.0,10.8,32.0,32.0,10.8,63.1,-11.9,8.51,8.51,-11.9,23.0,-23.5,-25.1,-25.1,-23.5,142.0,42.8,-25.3,-25.3,42.8,9.42,-5.82,1.67,1.67,-5.82,23.5,-9.37,-30.1,-30.1,-9.37,84.2,12.5,31.4,31.4,12.5,78.6,3.33,-42.0,-42.0,3.33,70.5,-21.3,-55.1,-55.1,-21.3,167.0,20.1,-15.6,-15.6,20.1,78.2,-3.88,-47.8,-47.8,-3.88,76.7,-7.38,-68.2,-68.2,-7.38,124.0,17.3,-12.6,-12.6,17.3,80.9,-3.54,-43.3,-43.3,-3.54,69.4,-5.94,-52.4,-52.4,-5.94,154.0,21.1,2.69,2.69,21.1,127.0,4.01,-60.5,-60.5,4.01,43.5,-16.0,-27.6,-27.6,-16.0,122.0,18.6,-4.41,-4.41,18.6,36.5,-9.72,-9.71,-9.71,-9.72,76.1,-9.38,-42.7,-42.7,-9.38,91.1,20.9,31.4,31.4,20.9,36.1,-17.3,-6.34,-6.34,-17.3,25.9,-6.66,-20.0,-20.0,-6.66,78.7,9.03,46.8,46.8,9.03,102.0,7.12,-39.3,-39.3,7.12,101.0,4.97,-85.5,-85.5,4.97,135.0,27.7,-25.4,-25.4,27.7,90.5,8.06,-65.6,-65.6,8.06,48.5,-0.135,-47.6,-47.6,-0.135,178.0,29.6,-43.3,-43.3,29.6,56.9,-17.9,-0.361,-0.361,-17.9,36.5,-20.1,-35.0,-35.0,-20.1,159.0,25.7,-29.9,-29.9,25.7,107.0,-5.8,-73.5,-73.5,-5.8,130.0,7.56,-114.0,-114.0,7.56,187.0,32.3,-58.8,-58.8,32.3,64.6,-19.8,-23.0,-23.0,-19.8,96.3,-11.9,-51.8,-51.8,-11.9,135.0,16.7,-15.2,-15.2,16.7,45.7,-15.1,-48.4,-48.4,-15.1,10.7,-20.3,-31.4,-31.4,-20.3,72.5,-1.27,31.9,31.9,-1.27,95.2,-13.1,-42.0,-42.0,-13.1,77.0,-11.0,-58.7,-58.7,-11.0,156.0,5.65,-6.67,-6.67,5.65,50.9,-19.6,-27.8,-27.8,-19.6,74.0,-10.1,-64.2,-64.2,-10.1,98.5,25.8,5.88,5.88,25.8,45.2,-9.45,-18.7,-18.7,-9.45,41.6,-19.5,-34.8,-34.8,-19.5,148.0,13.0,-25.9,-25.9,13.0,30.6,-26.2,-32.9,-32.9,-26.2,43.1,-26.7,-25.3,-25.3,-26.7,52.5,-8.02,53.3,53.3,-8.02,63.5,-20.9,-28.3,-28.3,-20.9,37.5,-12.4,-32.6,-32.6,-12.4,144.0,24.7,-1.86,-1.86,24.7,32.9,9.96,-27.9,-27.9,9.96,25.1,6.05,-5.71,-5.71,6.05,188.0,60.0,-53.9,-53.9,60.0,70.9,5.47,-34.8,-34.8,5.47,48.9,-9.81,-42.1,-42.1,-9.81,140.0,29.4,-17.4,-17.4,29.4,99.6,-3.73,-74.7,-74.7,-3.73,37.8,-20.1,-31.2,-31.2,-20.1,143.0,26.4,-16.0,-16.0,26.4,82.9,-9.03,-36.8,-36.8,-9.03,70.1,-4.25,-47.8,-47.8,-4.25,99.6,11.2,41.7,41.7,11.2"

            col1, col2, col3 = st.columns(3)

            if 'pasted_emotion_data' not in st.session_state:
                st.session_state.pasted_emotion_data = ""

            with col1:
                if st.button("😊"):
                    st.session_state.pasted_emotion_data = happy_sample

            with col2:
                if st.button("😐"):
                    st.session_state.pasted_emotion_data = neutral_sample

            with col3:
                if st.button("😠"):
                    st.session_state.pasted_emotion_data = angry_sample

            # Text area for pasting data
            pasted_data = st.text_area(
                "Paste a row of data (comma-separated values) to get emotion predictions.",
                value=st.session_state.pasted_emotion_data,
                height=100
            )

            predict_button = st.button("Predict Emotion")
            prediction_result_placeholder = st.empty()

        # Create a container for visualization
        visualization_container = st.container()

        if pasted_data and predict_button:
            try:
                values = [float(x.strip()) for x in pasted_data.split(',')]
                # Use your actual feature names here if possible!
                feature_names = [f'Feature_{i}' for i in range(len(values))]

                if len(values) == len(feature_names):
                    input_data = dict(zip(feature_names, values))
                    input_df = pd.DataFrame([input_data])

                    # Preprocess the data
                    X = preprocess_emotion_data(input_df)

                    # Make emotion prediction
                    prediction = make_emotion_predictions(emotion_model, emotion_scaler, feature_selector, X)

                    if prediction is not None:
                        predicted_label = prediction[0]
                        result = emotion_labels.get(predicted_label, "Unknown Emotion")

                        # Display prediction result
                        with prediction_result_placeholder.container():
                            st.subheader("Emotion Prediction Result")
                            st.write(f"Predicted Label: {predicted_label}")
                            result_html = f"""
                            <div style="font-size: 1.2em; margin-top: 5px; color: {emotion_colors[predicted_label]};">
                                <strong>Emotion:</strong> {result}
                            </div>
                            """
                            st.markdown(result_html, unsafe_allow_html=True)
                            st.markdown("---")

                    # # Create emotion visualization
                    # with visualization_container:
                    #     st.subheader("Emotion Visualization")
                    #     emotion_counts = {predicted_label: 1}
                    #     for emotion in emotion_labels.keys():
                    #         if emotion != predicted_label:
                    #             emotion_counts[emotion] = 0
                    #     gauge_fig = create_emotion_gauge(emotion_counts)
                    #     st.plotly_chart(gauge_fig, use_container_width=True)

                    #     st.subheader("Signal Features")
                    #     fig = go.Figure()
                    #     fig.add_trace(go.Scatter(
                    #         y=list(input_data.values()),
                    #         mode='lines+markers',
                    #         name='EEG Signal',
                    #         line=dict(color=emotion_colors[predicted_label], width=2),
                    #         marker=dict(size=5, color=emotion_colors[predicted_label])
                    #     ))
                    #     fig.update_layout(
                    #         title='Input EEG Signal',
                    #         xaxis_title='Feature',
                    #         yaxis_title='Value',
                    #         height=300,
                    #         margin=dict(l=10, r=10, t=40, b=80),
                    #         xaxis=dict(
                    #             ticktext=list(input_data.keys()),
                    #             tickvals=list(range(len(input_data))),
                    #             tickangle=45,
                    #             tickfont=dict(size=8)
                    #         )
                    #     )
                    #     st.plotly_chart(fig, use_container_width=True)

                else:
                    st.error(f"Expected {len(feature_names)} values, got {len(values)}")
            except Exception as e:
                st.error(f"Error processing pasted data: {str(e)}")

    with tab5:
        # Robotic Arm EEG Batch Prediction content
        robotic_batch_cols = st.columns([7, 3])

        with robotic_batch_cols[0]:
            st.header("Robotic Arm EEG Batch Prediction")
            st.write("Upload a CSV file containing EEG signals for robotic arm batch prediction.")

            # Add sample data download
            st.markdown("### Sample Data")
            try:
                sample_data = pd.read_csv('sample_batch_data.csv')
                st.markdown(get_table_download_link(sample_data, 'sample_batch_data.csv'), unsafe_allow_html=True)
            except Exception as e:
                st.error(f"Error loading sample data: {str(e)}")

            uploaded_file = st.file_uploader("Choose a CSV file for Robotic Arm EEG", type="csv", key="robotic_eeg_uploader")

            if uploaded_file is not None:
                try:
                    df = pd.read_csv(uploaded_file)
                    st.write("Preview of uploaded data:")
                    st.dataframe(df.head())

                    X = preprocess_eeg_data(df)
                    if st.button("Make Robotic Arm EEG Predictions"):
                        with st.spinner("Processing EEG signals for robotic arm..."):
                            predictions = make_eeg_predictions(eeg_model, eeg_scaler, X)

                            if predictions is not None:
                                results_df = df.copy()
                                results_df['Predicted_Label'] = predictions
                                results_df['Command'] = results_df['Predicted_Label'].map(robotic_arm_commands)

                                st.subheader("Robotic Arm EEG Prediction Results")
                                st.dataframe(results_df)

                                csv = results_df.to_csv(index=False)
                                st.download_button(
                                    label="Download Robotic Arm EEG Predictions",
                                    data=csv,
                                    file_name='robotic_arm_eeg_predictions.csv',
                                    mime='text/csv'
                                )

                                st.subheader("Prediction Distribution")
                                fig = px.histogram(results_df, x='Command',
                                                 title='Distribution of Predicted Robotic Arm Commands')
                                st.plotly_chart(fig)
                            else:
                                st.error("Failed to make predictions. Please try again.")

                except Exception as e:
                    st.error(f"Error processing file: {str(e)}")

        # Right column - EEG Sensor Positions
        with robotic_batch_cols[1]:
            st.markdown("""
            <h2 style="font-size: 1.8rem; font-weight: 600; margin-bottom: 0.5rem;">EEG Sensor Positions</h2>
            """, unsafe_allow_html=True)

            with st.container():
                st.markdown("""
                <div class="sensor-container">
                    <div class="sensor-subtitle">International 10-20 System</div>
                """, unsafe_allow_html=True)

                if eeg_sensor_image:
                    st.image(
                        eeg_sensor_image,
                        output_format="PNG",
                        width=250
                    )

                st.markdown("""
                    <div class="sensor-description">
                        Key positions shown: Frontal (F), Central (C), Parietal (P), Occipital (O), and Temporal (T) regions
                    </div>
                </div>
                """, unsafe_allow_html=True)

                with st.expander("About EEG Sensors"):
                    st.markdown("""
                    ### Frontal Region
                    - **Fp1, Fp2**: Prefrontal cortex (attention, planning)
                    - **F3, F4**: Frontal lobe (motor planning, emotional regulation)

                    ### Central Region
                    - **C1, C2**: Central region (fine motor control)
                    - **C3, C4**: Central region (sensorimotor functions)
                    - **Cz**: Central midline (motor coordination, supplementary motor area)

                    ### Centro-Parietal Region
                    - **CP1, CP2**: Transition between central and parietal regions
                    - **CP3, CP4**: Sensorimotor integration areas

                    ### Parietal Region
                    - **P3, P4**: Parietal lobe (sensory integration)
                    - **Pz**: Parietal midline (attention, perception)

                    ### Occipital Region
                    - **O1, O2**: Occipital lobe (visual processing)

                    The "z" (zero) indicates electrodes placed along the midline of the head.
                    Odd numbers (1, 3) indicate positions on the left hemisphere, while
                    even numbers (2, 4) indicate positions on the right hemisphere.
                    """)

    with tab6:
        st.header("Robotic Arm Single Signal Prediction")

        input_col = st.container()

        with input_col:
            st.write("Click the below buttons to get predictions.")

            col1, col2, col3, col4 = st.columns(4)

            if 'pasted_data' not in st.session_state:
                st.session_state.pasted_data = ""

            with col1:
                if st.button("👊"):
                    st.session_state.pasted_data = "17.895,18.892,0.5593599999999999,4.19,1.3399031272446529,1.451972451529298,0.0320338884493001,0.2706543858059262,6.720000000000001,76.88,140.49172965028924,242.71788485255345,14.893473125791424,92.5938578955408,158.00432313596815,69.92,17.648413440914005,17.60878433820803,15.942900926788193,41.39375918963057,31.65929981399516,28.51668016333828,11.837738400891023,44.59684556811312,23.26107343091225,31.31308591692754,26.198424253335173,15.122300481001137,11.947044300968017,35.65477050898056,40.74540871709828,3.170204831248526,17.743787859483145,7.832579625232223,14.202732782976067,11.234295213345131,25.86605384164161,10.297599158678702,15.900881342063846,10.847474399616363,1.2189751711523162,6.250109488078362,7.004108841778587,2.070034778184759,3.987326881330624,1.6221966296951873,1.8513863201267733,1.0579632953423768,0.6048262688858255,1.1045783463260963,0.2327169453534671,0.1513056928571692,0.5569245010654496,0.3110401781217685,0.5506589456930167,0.4080679794684767,0.1097516947495553,0.1624563203280561,0.3824027368042154,0.2869173274904243,0.497512987331642,0.323380254486268,0.2630683124703888,0.4477154213611007,0.4652196010429258,0.5571351111841539,0.0248571056015484,0.2282924971433172,0.2121943384031659,0.39426722460893,0.1437967244183503,0.3457206082414431,0.0580132488613787,0.1442489900632044,0.3821711343209582,0.2631918582884829,0.4013443380081823,0.3481501876481251,0.3481501876481251,0.4013443380081823,0.2631918582884829,0.3821711343209582,0.1442489900632044,0.0580132488613787,0.3457206082414431,0.1437967244183503,0.39426722460893,0.2121943384031659,0.2282924971433172,0.0248571056015484,0.5571351111841539,0.4652196010429258,0.4477154213611007,0.2630683124703888,0.323380254486268,0.497512987331642,0.2869173274904243,0.3824027368042154,0.1624563203280561,0.1097516947495553,0.4080679794684767,0.5506589456930167,0.3110401781217685,0.5569245010654496,0.1513056928571692,0.2327169453534671,1.1045783463260963,0.6048262688858255,1.0579632953423768,1.8513863201267733,1.6221966296951873,3.987326881330624,2.070034778184759,7.004108841778587,6.250109488078362,1.2189751711523162,10.847474399616363,15.900881342063846,10.297599158678702,25.86605384164161,11.234295213345131,14.202732782976067,7.832579625232223,17.743787859483145,3.170204831248526,40.74540871709828,35.65477050898056,11.947044300968017,15.122300481001137,26.198424253335173,31.31308591692754,23.26107343091225,44.59684556811312,11.837738400891023,28.51668016333828,31.65929981399516,41.39375918963057,15.942900926788193,17.60878433820803,17.648413440914005"

            with col2:
                if st.button("✋"):
                    st.session_state.pasted_data = "12.941,13.925,0.6384,10.47,5.69507750956912,5.730747106617077,-0.6799478080579413,-0.5096076385895788,23.78,161.44000000000003,283.2301102543645,535.8684057938913,122.84759251428356,757.493569136136,253.4486183671476,79.8,131.81576884959443,194.79125209214985,370.9546482454237,59.93189994946399,89.09799810426821,55.24933294773325,24.86027344873845,24.309113916943712,100.42302432769688,90.32092342764348,10.85806718874554,57.31898139333484,47.020354909146,67.64011704806995,18.10636717029908,78.98041804595647,30.3558967577572,59.4786643680694,10.922036293820709,44.5779904652658,29.437622539853944,36.05712785609055,14.347660669690088,5.792537390504529,6.797032729507975,11.944572227237176,2.103552362594324,4.660091628087092,4.62346371107315,5.70391822753298,3.925774688274664,3.700859356470557,4.056848371028937,4.741094330910339,4.563123522756883,4.119646171547212,3.814004389982326,3.890024903357447,3.481816369701896,3.9747004808916495,3.750845093219823,4.061976044838742,3.557365181930646,3.682522269370368,3.3782282674037063,3.768286880229158,3.4267676044531163,3.465072273168813,3.767530754360707,3.0512859705372515,3.563266785166551,3.5229448100731555,3.5044338404928923,3.299954363057375,3.1170446627813537,3.4812779750409564,3.2402856971806333,3.2599130450828198,3.581453526862449,3.6143937199739455,3.451437312299841,3.329495624304338,3.329495624304338,3.451437312299841,3.6143937199739455,3.581453526862449,3.2599130450828198,3.2402856971806333,3.4812779750409564,3.1170446627813537,3.299954363057375,3.5044338404928923,3.5229448100731555,3.563266785166551,3.0512859705372515,3.767530754360707,3.465072273168813,3.4267676044531163,3.768286880229158,3.3782282674037063,3.682522269370368,3.557365181930646,4.061976044838742,3.750845093219823,3.9747004808916495,3.481816369701896,3.890024903357447,3.814004389982326,4.119646171547212,4.563123522756883,4.741094330910339,4.056848371028937,3.700859356470557,3.925774688274664,5.70391822753298,4.62346371107315,4.660091628087092,2.103552362594324,11.944572227237176,6.797032729507975,5.792537390504529,14.347660669690088,36.05712785609055,29.437622539853944,44.5779904652658,10.922036293820709,59.4786643680694,30.3558967577572,78.98041804595647,18.10636717029908,67.64011704806995,47.020354909146,57.31898139333484,10.85806718874554,90.32092342764348,100.42302432769688,24.309113943712,24.86027344873845,55.24933294773325,89.09799810426821,59.93189994946399,370.9546482454237,194.79125209214985,131.81576884959443"

            with col3:
                if st.button("↪️"):
                    st.session_state.pasted_data = "17.395,18.388,0.42024,5.22,1.6131343224914656,1.6669745049040192,0.5025110070067309,0.239533240624632,9.1,92.08,165.4295971944718,309.57432302160333,58.24719610414238,84.92850314212005,172.0875267674643,52.53,4.349501719306558,4.28840771540847,39.862175538162425,36.4284181692426,30.46551890972791,25.01970495571268,46.3652031959139,33.808681536867226,38.296132245945,12.626765084859084,49.2025937833613,31.4954245434392,52.86057975123696,24.55065196899352,23.27063818626752,32.13744783420577,21.91764509090947,23.78136989731783,15.568329716878544,8.14584129140285,14.028810047107587,25.893231464828432,7.516908722403525,2.595273725776378,5.048236693896485,6.387403956953162,4.361018630229035,2.177201992767261,5.341293038851984,2.497016468137817,3.344512943382261,4.280914528765986,1.9053203907837477,2.2983513828834825,1.3831502596286656,2.00990638700909,1.5212893856644114,2.0075263733257755,2.1062263604341616,1.6701857137564855,1.4578214827166478,1.9360366404920484,1.5368561570693156,1.735988854665956,1.5867914113761434,1.584388961931582,1.7926898224178711,1.4874799944259034,2.1135601963672013,1.3390691850674177,1.361720566377774,1.2895449036330602,1.342126487978718,1.4518638252653944,1.6633728850614524,1.5582813919379872,1.4686284837006685,1.42874847968582,1.085152993841838,1.1811843939986342,1.3089963739920012,1.5124924183670656,1.5124924183670656,1.3089963739920012,1.1811843939986342,1.085152993841838,1.42874847968582,1.4686284837006685,1.5582813919379872,1.6633728850614524,1.4518638252653944,1.342126487978718,1.2895449036330602,1.361720566377774,1.3390691850674177,2.1135601963672013,1.4874799944259034,1.7926898224178711,1.584388961931582,1.5867914113761434,1.735988854665956,1.5368561570693156,1.9360366404920484,1.4578214827166478,1.6701857137564855,2.1062263604341616,2.0075263733257755,1.5212893856644114,2.00990638700909,1.3831502596286656,2.2983513828834825,1.9053203907837477,4.280914528765986,3.344512943382261,2.497016468137817,5.341293038851984,2.177201992767261,4.361018630229035,6.387403956953162,5.048236693896485,2.595273725776378,7.516908722403525,25.893231464828432,14.028810047107587,8.14584129140285,15.568329716878544,23.78136989731783,21.91764509090947,32.13744783420577,23.27063818626752,24.55065196899352,52.86057975123696,31.4954245434392,49.2025937833613,12.626765084859084,38.296132245945,33.808681536867226,46.3652031959139,25.01970495571268,30.46551890972791,36.4284181692426,39.862175538162425,4.28840771540847,4.349501719306558"

            with col4:
                if st.button("↩️"):
                    st.session_state.pasted_data = "13.925,14.924,1.87152,10.78,3.1565239250796115,3.669636276254092,0.1211691859223678,0.6603649862081264,14.81,132.56,198.80622450046425,407.0805420038367,41.30326758504787,377.607150917067,329.6929704450757,233.94,148.7013816289654,85.91753652415252,69.78750390743814,73.20072885651093,73.1322894014404,98.02996518368482,34.041009405378574,51.28897759806096,37.05852623857877,42.71157200488174,4.121505249611095,63.62564340933166,31.534858757085843,26.046172390334792,31.967841793106,35.26869307443615,16.832909786511177,14.58838382535911,72.71765423745356,39.38033547082446,13.050879655426137,10.622103786120103,16.61259155569531,7.327163401325991,2.697414899833233,9.808442155880304,4.763893431803291,1.933088662754356,5.737719153982554,2.5647525565726794,2.712132672587128,1.7239571826365292,1.6810039195561426,1.2736319462766283,2.1208764527542963,1.431964899448816,1.5971458512336798,1.2528988180493683,0.9752943870497016,1.3979536027060762,1.1679119156365092,1.1380472712930905,1.0470923307776576,0.8319348985150075,1.05327854458485,1.023053339527435,1.0215374156567398,1.2524345353520694,1.1490328654912667,1.001824764196653,1.068374193811373,1.0746584124407246,1.130687521124672,0.93008616766653,1.0557527635382242,0.6299639484264786,0.5573043439644885,0.7358360777936935,1.1573668875401568,1.3547238103124235,1.0282593944166858,1.1624938941100904,1.1624938941100904,1.0282593944166858,1.3547238103124235,1.1573668875401568,0.7358360777936935,0.5573043439644885,0.6299639484264786,1.0557527635382242,0.93008616766653,1.130687521124672,1.0746584124407246,1.068374193811373,1.001824764196653,1.1490328654912667,1.2524345353520694,1.0215374156567398,1.023053339527435,1.05327854458485,0.8319348985150075,1.0470923307776576,1.1380472712930905,1.1679119156365092,1.3979536027060762,0.9752943870497016,1.2528988180493683,1.5971458512336798,1.431964899448816,2.1208764527542963,1.2736319462766283,1.6810039195561426,1.7239571826365292,2.712132672587128,2.5647525565726794,5.737719153982554,1.933088662754356,4.763893431803291,9.808442155880304,2.697414899833233,7.327163401325991,16.61259155569531,10.622103786120103,13.050879655426137,39.38033547082446,72.71765423745356,14.58838382535911,16.832909786511177,35.26869307443615,31.967841793106,26.046172390334792,31.534858757085843,63.62564340933166,4.121505249611095,42.71157200488174,37.05852623857877,51.28897759806096,34.041009405378574,98.02996518368482,73.1322894014404,73.20072885651093,69.78750390743814,85.91753652415252,148.7013816289654"

            pasted_data = st.text_area("Paste a row of data (comma-separated values) to get robotic arm EEG predictions.", value=st.session_state.pasted_data, height=100)

            predict_button = st.button("Predict", key="predict_robotic_arm_single")

            prediction_result_placeholder = st.empty()

        visualization_container = st.container()

        if pasted_data and predict_button:
            try:
                values = [float(x.strip()) for x in pasted_data.split(',')]
                feature_names = ['Start Timestamp', 'End Timestamp', 'Mean', 'Max', 'Standard Deviation', 'RMS',
                                'Kurtosis', 'Skewness', 'Peak-to-Peak', 'Abs Diff Signal', 'Alpha Power',
                                'Beta Power', 'Gamma Power', 'Delta Power', 'Theta Power'] + \
                                [f'FFT_{i}' for i in range(125)]

                if len(values) == len(feature_names):
                    input_data = dict(zip(feature_names, values))
                    input_df = pd.DataFrame([input_data])

                    prediction = make_predictions(eeg_model, eeg_scaler, preprocess_eeg_data(input_df))

                    with prediction_result_placeholder.container():
                        st.subheader("Prediction Result")
                        predicted_label = prediction[0]
                        command = robotic_arm_commands.get(predicted_label, "Unknown Command")

                        st.write(f"Predicted Label: {predicted_label}")
                        command_html = f"""
                        <div style="font-size: 1.2em; margin-top: 5px;">
                            <strong>Command:</strong> {command}
                        </div>
                        """
                        st.markdown(command_html, unsafe_allow_html=True)
                        st.markdown("---")

                    with visualization_container:
                        viz_cols = st.columns([3, 7])

                        with viz_cols[0]:
                            st.subheader("Robotic Arm Visualization")

                            css = """
                            <style>
                            @keyframes moveToGrip {
                                0% { transform: translate(0, 0); }
                                50% { transform: translate(0, -80px); }
                                100% { transform: translate(0, 0); }
                            }
                            @keyframes moveToRelease {
                                0% { transform: translate(0, 0); }
                                50% { transform: translate(0, 80px); }
                                100% { transform: translate(0, 0); }
                            }
                            @keyframes rotateLeft {
                                0% { transform: rotate(0deg); }
                                50% { transform: rotate(-90deg); }
                                100% { transform: rotate(0deg); }
                            }
                            @keyframes rotateRight {
                                0% { transform: rotate(0deg); }
                                50% { transform: rotate(90deg); }
                                100% { transform: rotate(0deg); }
                            }
                            @keyframes glow {
                                0% { box-shadow: 0 0 5px rgba(0, 255, 0, 0.5); }
                                50% { box-shadow: 0 0 20px rgba(0, 255, 0, 0.8); }
                                100% { box-shadow: 0 0 5px rgba(0, 255, 0, 0.5); }
                            }
                            .wheel-container {
                                position: relative;
                                width: 300px;
                                height: 300px;
                                margin: 0;
                                border: 2px solid #e0e0e0;
                                border-radius: 15px;
                                background: linear-gradient(135deg, #e6f0fa 0%, #f0f8ff 100%);
                                overflow: hidden;
                                display: flex;
                                justify-content: center;
                                align-items: center;
                                box-shadow: inset 0 0 10px rgba(0, 0, 0, 0.1);
                            }
                            .wheel {
                                font-size: 5em;
                                color: #1a3c34;
                                text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.3);
                                transition: all 0.3s ease;
                            }
                            .move-to-grip { animation: moveToGrip 1.5s ease-in-out infinite, glow 1.5s ease-in-out infinite; }
                            .move-to-release { animation: moveToRelease 1.5s ease-in-out infinite, glow 1.5s ease-in-out infinite; }
                            .rotate-left { animation: rotateLeft 1.5s ease-in-out infinite, glow 1.5s ease-in-out infinite; }
                            .rotate-right { animation: rotateRight 1.5s ease-in-out infinite, glow 1.5s ease-in-out infinite; }
                            .arrow {
                                position: absolute;
                                font-size: 1.5em;
                                color: #4682b4;
                                opacity: 0.5;
                            }
                            .arrow-label {
                                font-size: 0.8em;
                                color: #2f4f4f;
                                font-weight: bold;
                            }
                            .grip-arrow { top: 10%; left: 50%; transform: translateX(-50%); }
                            .release-arrow { bottom: 10%; left: 50%; transform: translateX(-50%); }
                            .rotate-left-arrow { top: 50%; left: 10%; transform: translateY(-50%); }
                            .rotate-right-arrow { top: 50%; right: 10%; transform: translateY(-50%); }
                            .status-bar {
                                position: absolute;
                                bottom: 10px;
                                width: 80%;
                                height: 20px;
                                background: #d3d3d3;
                                border-radius: 10px;
                                overflow: hidden;
                            }
                            .status-fill {
                                height: 100%;
                                background: linear-gradient(90deg, #32cd32, #00ff00);
                                transition: width 1s ease;
                            }
                            </style>
                            """

                            animation_class = ""
                            status_percentage = 100  # For simplicity, full status on action
                            if predicted_label == 0:  # Grip
                                animation_class = "move-to-grip"
                            elif predicted_label == 1:  # Release
                                animation_class = "move-to-release"
                            elif predicted_label == 2:  # Rotate Left
                                animation_class = "rotate-left"
                            elif predicted_label == 3:  # Rotate Right
                                animation_class = "rotate-right"

                            html = f"""
                            {css}
                            <div class="wheel-container">
                                <div class="arrow grip-arrow">👊<br><span class="arrow-label">Grip</span></div>
                                <div class="arrow release-arrow">✋<br><span class="arrow-label">Release</span></div>
                                <div class="arrow rotate-left-arrow">↪️<br><span class="arrow-label">Rotate Left</span></div>
                                <div class="arrow rotate-right-arrow">↩️<br><span class="arrow-label">Rotate Right</span></div>
                                <div class="wheel {animation_class}">🦾</div>
                                <div class="status-bar">
                                    <div class="status-fill" style="width: {status_percentage}%;"></div>
                                </div>
                            </div>
                            """

                            st.markdown(html, unsafe_allow_html=True)

                        with viz_cols[1]:
                            st.subheader("Signal Visualization")

                            fig = go.Figure()
                            fig.add_trace(go.Scatter(
                                y=list(input_data.values())[2:],
                                mode='lines+markers',
                                name='EEG Signal',
                                line=dict(color='#3498db', width=2),
                                marker=dict(size=5, color='#2c3e50')
                            ))

                            fig.update_layout(
                                title='Input EEG Signal',
                                xaxis_title='Feature',
                                yaxis_title='Value',
                                height=300,
                                margin=dict(l=10, r=10, t=40, b=80),
                                xaxis=dict(
                                    ticktext=list(input_data.keys())[2:],
                                    tickvals=list(range(len(input_data)-2)),
                                    tickangle=45,
                                    tickfont=dict(size=8)
                                )
                            )

                            st.plotly_chart(fig, use_container_width=True)
                else:
                    st.error(f"Expected {len(feature_names)} values, got {len(values)}")
            except Exception as e:
                st.error(f"Error processing pasted data: {str(e)}")

if __name__ == "__main__":
    main()

Overwriting streamlit_app.py


In [2]:
!ls


best_eeg_model.joblib	   emotion_feature_selector.joblib  sample_emotion.csv
best_emotion_model.joblib  emotion_scaler.joblib	    streamlit_app.py
eeg_app.py		   sample_batch_data.csv
eeg_scaler.joblib	   sample_data


In [3]:
!pip install -q streamlit pyngrok nest_asyncio


In [4]:
import nest_asyncio
nest_asyncio.apply()
!ngrok authtoken 2tQHMWWX8EuRduhHUpO4VDCigRT_2kHzzHnYhTCucWSHCK99Y


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
from pyngrok import ngrok
ngrok.kill()
!streamlit run streamlit_app.py --server.port 8501 >/dev/null 2>&1 &
public_url = ngrok.connect(8501)
print("Streamlit app URL:", public_url.public_url)

Streamlit app URL: https://41bb-34-138-175-157.ngrok-free.app
